# **I - Mô hình Transformer Cơ bản**

**Bước 1: Cài đặt môi trường và thư viện**

In [ ]:
# Chạy ô này ĐẦU TIÊN và DUY NHẤT để cài đặt/nâng cấp
# Lệnh này sẽ cài đặt PyTorch, Torchvision và các thư viện liên quan với phiên bản tương thích
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Nâng cấp các thư viện cần thiết khác
!pip install --upgrade transformers accelerate sentencepiece googletrans sacrebleu

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s et

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
import numpy as np
from googletrans import Translator
import sacrebleu
from datasets import load_dataset
import spacy
#from lime.lime_text import LimeTextExplainer
import warnings
from google.colab import drive
import os
warnings.filterwarnings('ignore')

# Kết nối với Google Drive
drive.mount('/content/drive')

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Mounted at /content/drive
Using device: cuda


**Bước 2: Tải và tiền xử lý dữ liệu**

In [ ]:
# Hàm đọc và ghép cặp dữ liệu từ Drive
def load_split_data_from_drive(en_file, vi_or_th_file, limit=None, drive_path="/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/"):
    try:
        with open(drive_path + en_file, 'r', encoding='utf-8') as f_en, open(drive_path + vi_or_th_file, 'r', encoding='utf-8') as f_vi_th:
            en_lines = f_en.readlines()[:limit] if limit else f_en.readlines()
            vi_or_th_lines = f_vi_th.readlines()[:limit] if limit else f_vi_th.readlines()
            data = pd.DataFrame({"en": en_lines, "vi": vi_or_th_lines})
            return data
    except Exception as e:
        print(f"Lỗi đọc file: {e}")
        return None

# Đọc PhoMT từ Drive
train_phomt = load_split_data_from_drive("train_phomt.en", "train_phomt.vi", 120000)
val_phomt = load_split_data_from_drive("val_phomt.en", "val_phomt.vi", 15000)
test_phomt = load_split_data_from_drive("test_phomt.en", "test_phomt.vi", 15000)

# Đọc dữ liệu Thái từ Drive
#train_thai = load_split_data_from_drive("train_thai.en", "train_thai.th", 50000)
#val_thai = load_split_data_from_drive("val_thai.en", "val_thai.th", 5000)
#test_thai = load_split_data_from_drive("test_thai.en", "test_thai.th", 5000)

# Hàm tiền xử lý chung
def preprocess_text(text):
    return text.strip().lower()

train_phomt["en"] = train_phomt["en"].apply(preprocess_text)
train_phomt["vi"] = train_phomt["vi"].apply(preprocess_text)
val_phomt["en"] = val_phomt["en"].apply(preprocess_text)
val_phomt["vi"] = val_phomt["vi"].apply(preprocess_text)
test_phomt["en"] = test_phomt["en"].apply(preprocess_text)
test_phomt["vi"] = test_phomt["vi"].apply(preprocess_text)

# Khởi tạo tokenizer và mô hình
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_col="en", target_col="vi", max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.source_col = source_col
        self.target_col = target_col
        self.max_length = max_length
        self.source_texts = self.dataframe[self.source_col].tolist()
        self.target_texts = self.dataframe[self.target_col].tolist()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        # Tokenize on the fly
        model_inputs = self.tokenizer(source_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        # Để tương thích với mô hình, labels phải là input_ids
        model_inputs["labels"] = labels["input_ids"]

        # Squeeze để loại bỏ chiều batch không cần thiết (DataLoader sẽ thêm vào sau)
        return {key: val.squeeze() for key, val in model_inputs.items()}

**Bước 3: Huấn luyện mô hình Transformer cơ bản**

In [ ]:
import gc
import torch
import os
import gc
from tqdm import tqdm # Thêm tqdm để có thanh tiến trình đẹp mắt

def train_model_fully_optimized(model, train_df, val_df, tokenizer, batch_size=32, epochs=30,
                                checkpoint_dir="/content/drive/My Drive/TieuLuanNLP/model/basic_en_vi/checkpoints/",
                                accumulation_steps=4):

    # --- Thiết lập DataLoader ---
    train_dataset = TranslationDataset(train_df, tokenizer)
    # num_workers > 0 sẽ bật tính năng tải dữ liệu song song, tăng tốc đáng kể
    # pin_memory=True giúp chuyển dữ liệu từ CPU sang GPU nhanh hơn
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    # (Tùy chọn) Tạo DataLoader cho validation
    # val_dataset = TranslationDataset(val_df, tokenizer)
    # val_loader = DataLoader(val_dataset, batch_size=batch_size * 2) # Batch size có thể lớn hơn khi validation

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5) # Có thể thử learning rate khác

    os.makedirs(checkpoint_dir, exist_ok=True)
    start_epoch = 0
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pth")
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Tiếp tục từ epoch {start_epoch}")

    # ===== VÒNG LẶP HUẤN LUYỆN TỐI ƯU =====
    for epoch in range(start_epoch, epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()

        # Sử dụng tqdm để theo dõi tiến trình
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for step, batch in enumerate(progress_bar):
            # DataLoader đã tự động chuyển dữ liệu, chỉ cần đưa lên device
            inputs = {key: val.to(device) for key, val in batch.items()}

            outputs = model(**inputs)
            loss = outputs.loss
            loss = loss / accumulation_steps
            loss.backward()

            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            # Cập nhật thanh tiến trình với giá trị loss
            # Nhân lại loss với accumulation_steps để hiển thị đúng
            progress_bar.set_postfix({"loss": loss.item() * accumulation_steps})
            total_loss += loss.item() * accumulation_steps

            # Không cần giải phóng bộ nhớ quá thường xuyên khi đã dùng DataLoader
            # Chỉ thực hiện nếu vẫn gặp lỗi OOM
            if (step + 1) % 100 == 0:
              del inputs, outputs, loss
              gc.collect()
              torch.cuda.empty_cache()

        avg_train_loss = total_loss / len(train_loader)
        print(f"\nEpoch {epoch+1} - Average Training Loss: {avg_train_loss:.4f}")

        # --- (Đề xuất) Thêm vòng lặp Validation ở đây ---

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_train_loss,
        }, checkpoint_path)
        print(f"Đã lưu checkpoint tại epoch {epoch+1}")

        # Giải phóng bộ nhớ cuối mỗi epoch
        gc.collect()
        torch.cuda.empty_cache()

    print("Huấn luyện hoàn tất!")
    return model
def train_model(model, train_data, val_data, batch_size=16, epochs=30, checkpoint_dir="/content/drive/My Drive/TieuLuanNLP/model/basic_en_vi/checkpoints/", accumulation_steps=4):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    # Tạo thư mục checkpoint nếu chưa tồn tại
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Kiểm tra checkpoint hiện có
    start_epoch = 0
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pth")
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Tiếp tục từ epoch {start_epoch}")

    for epoch in range(start_epoch, epochs):
        model.train()
        total_loss = 0

        # Reset optimizer gradients lúc bắt đầu mỗi epoch
        optimizer.zero_grad()

        for i in range(0, len(train_data), batch_size):
            batch = train_data.iloc[i:i+batch_size]
            if batch["en"].isnull().any() or batch["vi"].isnull().any():
                print(f"Lỗi: Dữ liệu null tại batch {i}")
                continue
            inputs = tokenizer(batch["en"].tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
            labels = tokenizer(batch["vi"].tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)["input_ids"]
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss

            # --- TỐI ƯU 1: TÍCH LŨY GRADIENT ---
            # Chuẩn hóa loss theo số bước tích lũy
            loss = loss / accumulation_steps

            loss.backward()

            # Chỉ cập nhật trọng số sau accumulation_steps
            # (i // batch_size + 1) là số thứ tự của mini-batch hiện tại
            if (i // batch_size + 1) % accumulation_steps == 0:
                optimizer.step()      # Cập nhật trọng số mô hình
                optimizer.zero_grad() # Reset gradient cho lần tích lũy tiếp theo

            total_loss += loss.item() * accumulation_steps # Nhân lại để loss đúng với giá trị gốc
            # optimizer.step()
            # optimizer.zero_grad()
            # total_loss += loss.item()
            # --- TỐI ƯU 2: GIẢI PHÓNG BỘ NHỚ ---
            # Xóa các tensor không còn dùng đến sau mỗi bước
            if (i + 1) % 100 == 0:
              del inputs, labels, outputs, loss
              print(f"  Batch {i+1}/{len(train_data)}, Loss: {loss.item() * accumulation_steps:.4f}")
              torch.cuda.empty_cache()
              gc.collect()


        # Tính toán và in loss trung bình cho epoch
        # Lưu ý: total_loss đã được nhân lại với accumulation_steps ở trên
        avg_train_loss = total_loss / (len(train_data) / batch_size)
        print(f"Epoch {epoch+1}, Loss: {avg_train_loss}")

        # Lưu checkpoint sau mỗi epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss,
        }, checkpoint_path)
        print(f"Đã lưu checkpoint tại epoch {epoch+1}")

    return model

model_basic = MarianMTModel.from_pretrained(model_name, weights_only=False).to(device)
model_basic = train_model_fully_optimized(model_basic, train_phomt, val_phomt, tokenizer)
model_basic.save_pretrained("/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/model_basic")

pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

Tiếp tục từ epoch 28


Epoch 29/30: 100%|██████████| 3750/3750 [37:55<00:00,  1.65it/s, loss=0.134]



Epoch 29 - Average Training Loss: 0.1009
Đã lưu checkpoint tại epoch 29


Epoch 30/30: 100%|██████████| 3750/3750 [37:57<00:00,  1.65it/s, loss=0.109]



Epoch 30 - Average Training Loss: 0.0962
Đã lưu checkpoint tại epoch 30
Huấn luyện hoàn tất!


**Bước 4: Lưu trữ và sử dụng mô hình Transformer cơ bản**

In [ ]:
# Khởi tạo mô hình# Khởi tạo tokenizer và mô hình
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
#model_basic = MarianMTModel.from_pretrained(model_name).to(device)

# Huấn luyện
#model_basic = train_model(model_basic, train_phomt, val_phomt)

#model_basic.save_pretrained("/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/model_basic")

#Test model
model_basic =  MarianMTModel.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/basic_en_vi/checkpoints/")

sample_english = [
        "Brother Albert Barnett and his wife , Sister Susan Barnett , from the West Congregation in Tuscaloosa , Alabama",
        "Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12 , 2020 .",
        "Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .",
        "Sadly , Brother Albert Barnett and his wife , Sister Susan Barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .",
        "The United States branch also reports that at least four of our brothers' homes sustained minor damage , along with two Kingdom Halls .",
        "Additionally , the storms caused major damage to a brother 's business property .",
        "Local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster .",
        "We know that our heavenly Father , Jehovah , is providing comfort to our brothers and sisters who are grieving because of this tragedy .",
        "International government agencies and officials have responded to Russia 's Supreme Court decision that criminalizes the worship of Jehovah 's Witnesses in Russia .",
        "These statements have criticized Russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity ."
    ]
print(" BẮT ĐẦU KIỂM TRA MÔ HÌNH VỚI 10 CÂU VÍ DỤ ".center(80, "="))

# 2. Dịch từng câu và in kết quả
for i in range(len(sample_english)):
  text = sample_english[i]

  # Chuẩn bị input
  inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)  # Đảm bảo input trên CPU
  print(f"\n--- CÂU {i+1} ---")
  print(f"  > Tiếng Anh:".ljust(25), f"{text}")
  # Dự đoán
  outputs = model_basic.generate(**inputs)
  translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print("Translated text:", translated_text)

print("=" * 80)
print("✅ Hoàn tất kiểm tra.")

================== BẮT ĐẦU KIỂM TRA MÔ HÌNH VỚI 10 CÂU VÍ DỤ ===================

--- CÂU 1 ---
  > Tiếng Anh:            Brother Albert Barnett and his wife , Sister Susan Barnett , from the West Congregation in Tuscaloosa , Alabama
Translated text: Anh Albert Barnett và vợ Susan Barnett từ hội chính phương Tây ở Tuscaloosa, Alabama

--- CÂU 2 ---
  > Tiếng Anh:            Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12 , 2020 .
Translated text: Những cơn khó khăn xảy ra xuyên qua những phần của miền Nam và giữa Tây Nam Mỹ và ngày 11 / 1, 2020.

--- CÂU 3 ---
  > Tiếng Anh:            Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .
Translated text: Hai ngày có mưa, gió cao, và nhiều cơn lốc xoáy gây ra thiệt hại lớn ở nhiều bang.

--- CÂU 4 ---
  > Tiếng Anh:            Sadly , Brother Albert Barnett and his wife , Sister Susan Barnett , 85 and 75 years old respectively , wer

# **II - Mô hình Đa nguồn**

In [ ]:
# 1. Cài đặt các phiên bản thư viện cụ thể để đảm bảo tương thích
print("⚙️ Đang cài đặt các phiên bản thư viện ổn định...")
# Đóng băng phiên bản Numpy ở mức an toàn, tương thích với hầu hết các thư viện
!pip install numpy==1.26.4 --quiet
# Đóng băng phiên bản torch và torchtext
!pip install torch==2.2.0 torchtext==0.17.0 --quiet
# Đóng băng phiên bản spacy
!pip install spacy==3.7.2 --quiet
# Cài đặt các thư viện phụ trợ
!pip install portalocker sacrebleu pythainlp underthesea --quiet

# 2. Tải mô hình ngôn ngữ cho tiếng Anh (tương thích với spacy 3.7.2)
print("⚙️ Đang tải mô hình ngôn ngữ cho tiếng Anh...")
!python -m spacy download en_core_web_sm --quiet

print("✅ Cài đặt và tải mô hình hoàn tất!")

# ==============================================================================
# QUAN TRỌNG NHẤT: BẠN PHẢI KHỞI ĐỘNG LẠI RUNTIME SAU KHI CHẠY XONG CELL NÀY
# MENU "Runtime" -> "Restart runtime".
# VIỆC NÀY SẼ NẠP CÁC PHIÊN BẢN THƯ VIỆN CHÍNH XÁC MÀ CHÚNG TA VỪA CÀI.
# ==============================================================================

⚙️ Đang cài đặt các phiên bản thư viện ổn định...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7

In [ ]:
# 3. Import các thư viện
import numpy
import spacy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import math
import time
import os
from google.colab import drive
import sacrebleu
from pythainlp.tokenize import word_tokenize as th_word_tokenize
from underthesea import word_tokenize as vi_word_tokenize

print("✅ Import thư viện hoàn tất!")
print(f"Phiên bản Numpy đang dùng: {numpy.__version__}")
print(f"Phiên bản Torch đang dùng: {torch.__version__}")


# 4. Thiết lập Tokenizer và Vocab (Phần này giữ nguyên)

# Kết nối Google Drive và lấy đường dẫn file
drive.mount('/content/drive')
EN_FILE_PATH = '/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/train_thai.en'
TH_FILE_PATH = '/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/train_thai.th'
VI_FILE_PATH = '/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/train_thai.vi'

print("⚙️ Đang thiết lập tokenizer...")
nlp_en = spacy.load('en_core_web_sm')

def tokenize_en(text):
    return [tok.text for tok in nlp_en.tokenizer(text)]

def tokenize_th(text):
    return th_word_tokenize(text)

def tokenize_vi(text):
    return vi_word_tokenize(text)

print("✅ Tokenizer sẵn sàng!")

def build_vocabulary(filepath, tokenizer, specials=['<unk>', '<pad>', '<bos>', '<eos>']):
    def yield_tokens(filepath, tokenizer):
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                yield tokenizer(line.strip())

    vocab = build_vocab_from_iterator(yield_tokens(filepath, tokenizer), specials=specials)
    vocab.set_default_index(vocab['<unk>'])
    return vocab

print("⚙️ Đang xây dựng vocab...")
vocab_en = build_vocabulary(EN_FILE_PATH, tokenize_en)
vocab_th = build_vocabulary(TH_FILE_PATH, tokenize_th)
vocab_vi = build_vocabulary(VI_FILE_PATH, tokenize_vi)

print(f"Kích thước Vocab Anh: {len(vocab_en)}")
print(f"Kích thước Vocab Thái: {len(vocab_th)}")
print(f"Kích thước Vocab Việt: {len(vocab_vi)}")

# class TranslationDataset(Dataset):
#     def __init__(self, en_path, th_path, vi_path):
#         with open(en_path, 'r', encoding='utf-8') as f:
#             self.en_data = f.readlines()
#         with open(th_path, 'r', encoding='utf-8') as f:
#             self.th_data = f.readlines()
#         with open(vi_path, 'r', encoding='utf-8') as f:
#             self.vi_data = f.readlines()

#     def __len__(self):
#         return len(self.vi_data)

#     def __getitem__(self, idx):
#         return self.en_data[idx].strip(), self.th_data[idx].strip(), self.vi_data[idx].strip()
# Phiên bản TranslationDataset linh hoạt hơn
class TranslationDataset(Dataset):
    # Thay đổi tên tham số để trở nên tổng quát
    def __init__(self, src1_path, src2_path, tgt_path):
        with open(src1_path, 'r', encoding='utf-8') as f:
            self.src1_data = f.readlines()
        with open(src2_path, 'r', encoding='utf-8') as f:
            self.src2_data = f.readlines()
        with open(tgt_path, 'r', encoding='utf-8') as f:
            self.tgt_data = f.readlines()

    def __len__(self):
        # Dùng độ dài của file đích làm tham chiếu
        return len(self.tgt_data)

    def __getitem__(self, idx):
        # Trả về theo đúng thứ tự tổng quát
        return self.src1_data[idx].strip(), self.src2_data[idx].strip(), self.tgt_data[idx].strip()

PAD_IDX = vocab_en['<pad>']
BOS_IDX = vocab_en['<bos>']
EOS_IDX = vocab_en['<eos>']

# def collate_fn(batch):
#    en_batch, th_batch, vi_batch = [], [], []
#    for en_text, th_text, vi_text in batch:
#         en_tensor = torch.tensor([BOS_IDX] + [vocab_en[token] for token in tokenize_en(en_text)] + [EOS_IDX])
#         th_tensor = torch.tensor([BOS_IDX] + [vocab_th[token] for token in tokenize_th(th_text)] + [EOS_IDX])
#         vi_tensor = torch.tensor([BOS_IDX] + [vocab_vi[token] for token in tokenize_vi(vi_text)] + [EOS_IDX])
#         en_batch.append(en_tensor)
#         th_batch.append(th_tensor)
#         vi_batch.append(vi_tensor)

#     en_padded = pad_sequence(en_batch, batch_first=True, padding_value=PAD_IDX)
#     th_padded = pad_sequence(th_batch, batch_first=True, padding_value=PAD_IDX)

#     max_len = max(en_padded.size(1), th_padded.size(1))

#     if en_padded.size(1) < max_len:
#         pad_needed = max_len - en_padded.size(1)
#         en_padded = F.pad(en_padded, (0, pad_needed), 'constant', PAD_IDX)
#     if th_padded.size(1) < max_len:
#         pad_needed = max_len - th_padded.size(1)
#         th_padded = F.pad(th_padded, (0, pad_needed), 'constant', PAD_IDX)

#     vi_padded = pad_sequence(vi_batch, batch_first=True, padding_value=PAD_IDX)

#     return en_padded, th_padded, vi_padded

def collate_fn(batch):
    vi_batch, th_batch, en_batch = [], [], []

    # Thay đổi thứ tự đọc dữ liệu: (vi_text, th_text) là nguồn, (en_text) là đích
    for vi_text, th_text, en_text in batch:
        vi_tensor = torch.tensor([BOS_IDX] + [vocab_vi[token] for token in tokenize_vi(vi_text)] + [EOS_IDX])
        th_tensor = torch.tensor([BOS_IDX] + [vocab_th[token] for token in tokenize_th(th_text)] + [EOS_IDX])
        en_tensor = torch.tensor([BOS_IDX] + [vocab_en[token] for token in tokenize_en(en_text)] + [EOS_IDX])
        vi_batch.append(vi_tensor)
        th_batch.append(th_tensor)
        en_batch.append(en_tensor)

    # Padding đồng bộ cho hai ngôn ngữ nguồn mới (Việt, Thái)
    vi_padded = pad_sequence(vi_batch, batch_first=True, padding_value=PAD_IDX)
    th_padded = pad_sequence(th_batch, batch_first=True, padding_value=PAD_IDX)

    max_src_len = max(vi_padded.size(1), th_padded.size(1))

    if vi_padded.size(1) < max_src_len:
        vi_padded = F.pad(vi_padded, (0, max_src_len - vi_padded.size(1)), 'constant', PAD_IDX)
    if th_padded.size(1) < max_src_len:
        th_padded = F.pad(th_padded, (0, max_src_len - th_padded.size(1)), 'constant', PAD_IDX)

    # Pad ngôn ngữ đích (Anh) một cách độc lập
    en_padded = pad_sequence(en_batch, batch_first=True, padding_value=PAD_IDX)

    return vi_padded, th_padded, en_padded

print("✅ Class Dataset và hàm collate đã được cập nhật!")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import os
from google.colab import drive
# Kết nối với Google Drive
drive.mount('/content/drive')

# --- THAY ĐỔI CÁC ĐƯỜNG DẪN NÀY CHO PHÙ HỢP ---
# Đường dẫn tới các file dữ liệu
EN_FILE_PATH = '/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/train_thai.en'
TH_FILE_PATH = '/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/train_thai.th'
VI_FILE_PATH = '/content/drive/My Drive/TieuLuanNLP/MT_Translation_Data/train_thai.vi'

# Đường dẫn để lưu checkpoint
CHECKPOINT_DIR = '/content/drive/MyDrive/TieuLuanNLP/model/multisource_vi_en'
if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

print("✅ Kết nối Google Drive và thiết lập đường dẫn hoàn tất!")
print(f"Thư mục lưu checkpoint: {CHECKPOINT_DIR}")


Mounted at /content/drive
✅ Kết nối Google Drive và thiết lập đường dẫn hoàn tất!
Thư mục lưu checkpoint: /content/drive/MyDrive/TieuLuanNLP/model/multisource_vi_en


In [ ]:
import torch
import torch.nn as nn
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        #====== SỬA LỖI TẠI ĐÂY ======
        # Đảm bảo max_len là kiểu int trước khi truyền vào torch.zeros
        pe = torch.zeros(int(max_len), d_model)
        #============================

        position = torch.arange(0, int(max_len), dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class MultiSourceTransformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, d_model, num_heads,
                 input_vocab_size_en, input_vocab_size_th, output_vocab_size_vi,
                 d_ff, dropout=0.1):
        super(MultiSourceTransformer, self).__init__()

        # --- Encoders ---
        self.embedding_en = nn.Embedding(input_vocab_size_en, d_model)
        self.embedding_th = nn.Embedding(input_vocab_size_th, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, d_ff, dropout, batch_first=True)
        self.transformer_encoder_en = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        self.transformer_encoder_th = nn.TransformerEncoder(encoder_layer, num_encoder_layers)

        # --- Decoder ---
        self.embedding_vi = nn.Embedding(output_vocab_size_vi, d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model, num_heads, d_ff, dropout, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers)

        # --- Lớp kết hợp 2 encoder output ---
        self.fusion_layer = nn.Linear(d_model * 2, d_model)

        # --- Lớp đầu ra ---
        self.fc_out = nn.Linear(d_model, output_vocab_size_vi)
        self.d_model = d_model

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def create_padding_mask(self, seq, pad_idx):
        return (seq == pad_idx)

    def forward(self, src_en, src_th, tgt):
        # Tạo mask
        src_en_padding_mask = self.create_padding_mask(src_en, PAD_IDX).to(device)
        src_th_padding_mask = self.create_padding_mask(src_th, PAD_IDX).to(device)
        tgt_padding_mask = self.create_padding_mask(tgt, PAD_IDX).to(device)
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(device)

        # --- Encoder Forward ---
        # Encoder cho tiếng Anh
        src_en_emb = self.pos_encoder(self.embedding_en(src_en) * math.sqrt(self.d_model))
        memory_en = self.transformer_encoder_en(src_en_emb, src_key_padding_mask=src_en_padding_mask)

        # Encoder cho tiếng Thái
        src_th_emb = self.pos_encoder(self.embedding_th(src_th) * math.sqrt(self.d_model))
        memory_th = self.transformer_encoder_th(src_th_emb, src_key_padding_mask=src_th_padding_mask)

        # --- Kết hợp (Fusion) ---
        memory_combined = torch.cat((memory_en, memory_th), dim=-1)
        memory = torch.tanh(self.fusion_layer(memory_combined))

        # --- Decoder Forward ---
        tgt_emb = self.pos_encoder(self.embedding_vi(tgt) * math.sqrt(self.d_model))
        memory_padding_mask = src_en_padding_mask

        output = self.transformer_decoder(tgt_emb, memory,
                                          tgt_mask=tgt_mask,
                                          tgt_key_padding_mask=tgt_padding_mask,
                                          memory_key_padding_mask=memory_padding_mask)

        return self.fc_out(output)
# class MultiSourceTransformer(nn.Module):
#     # Thay đổi tên tham số để trở nên tổng quát
#     def __init__(self, num_encoder_layers, num_decoder_layers, d_model, num_heads,
#                  input_vocab_size_src1, input_vocab_size_src2, output_vocab_size_tgt,
#                  d_ff, dropout=0.1):
#         super(MultiSourceTransformer, self).__init__()

#         # --- Encoders cho 2 ngôn ngữ nguồn ---
#         self.embedding_src1 = nn.Embedding(input_vocab_size_src1, d_model)
#         self.embedding_src2 = nn.Embedding(input_vocab_size_src2, d_model)
#         self.pos_encoder = PositionalEncoding(d_model, dropout)

#         encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, d_ff, dropout, batch_first=True)
#         self.transformer_encoder_src1 = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
#         self.transformer_encoder_src2 = nn.TransformerEncoder(encoder_layer, num_encoder_layers)

#         # --- Decoder cho ngôn ngữ đích ---
#         self.embedding_tgt = nn.Embedding(output_vocab_size_tgt, d_model)
#         decoder_layer = nn.TransformerDecoderLayer(d_model, num_heads, d_ff, dropout, batch_first=True)
#         self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers)

#         self.fusion_layer = nn.Linear(d_model * 2, d_model)
#         self.fc_out = nn.Linear(d_model, output_vocab_size_tgt)
#         self.d_model = d_model

#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask

#     def create_padding_mask(self, seq, pad_idx):
#         return (seq == pad_idx)

#     # Thay đổi tên tham số của hàm forward
#     def forward(self, src1, src2, tgt):
#         # Tạo mask
#         src1_padding_mask = self.create_padding_mask(src1, PAD_IDX).to(device)
#         src2_padding_mask = self.create_padding_mask(src2, PAD_IDX).to(device)
#         tgt_padding_mask = self.create_padding_mask(tgt, PAD_IDX).to(device)
#         tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(device)

#         # --- Encoder Forward ---
#         src1_emb = self.pos_encoder(self.embedding_src1(src1) * math.sqrt(self.d_model))
#         memory1 = self.transformer_encoder_src1(src1_emb, src_key_padding_mask=src1_padding_mask)

#         src2_emb = self.pos_encoder(self.embedding_src2(src2) * math.sqrt(self.d_model))
#         memory2 = self.transformer_encoder_src2(src2_emb, src_key_padding_mask=src2_padding_mask)

#         # --- Kết hợp (Fusion) ---
#         memory_combined = torch.cat((memory1, memory2), dim=-1)
#         memory = torch.tanh(self.fusion_layer(memory_combined))

#         # --- Decoder Forward ---
#         tgt_emb = self.pos_encoder(self.embedding_tgt(tgt) * math.sqrt(self.d_model))
#         memory_padding_mask = src1_padding_mask # Có thể dùng mask của src1 hoặc src2

#         output = self.transformer_decoder(tgt_emb, memory,
#                                           tgt_mask=tgt_mask,
#                                           tgt_key_padding_mask=tgt_padding_mask,
#                                           memory_key_padding_mask=memory_padding_mask)

#         return self.fc_out(output)

print("✅ Định nghĩa kiến trúc mô hình hoàn tất!")

✅ Định nghĩa kiến trúc mô hình hoàn tất!


In [ ]:
import gc
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import StepLR

# Thiết lập các tham số
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

# --- Các tham số có thể điều chỉnh ---
D_MODEL = 512       # Kích thước embedding
NUM_HEADS = 8       # Số lượng attention heads
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
D_FF = 2048         # Kích thước lớp FeedForward
DROPOUT = 0.1
BATCH_SIZE = 16     # Giảm batch size để tiết kiệm bộ nhớ, sẽ bù lại bằng Gradient Accumulation
LEARNING_RATE = 0.0001
EPOCHS = 30

# --- THAM SỐ TỐI ƯU MỚI ---
# Tích lũy gradient sau 4 bước. Batch size hiệu quả = 16 * 4 = 64
GRADIENT_ACCUMULATION_STEPS = 4
# Giảm LR đi 10% sau mỗi 2 epoch
LR_SCHEDULER_STEP_SIZE = 2
LR_SCHEDULER_GAMMA = 0.9

# Khởi tạo mô hình
# INPUT_VOCAB_SIZE_EN = len(vocab_en)
# INPUT_VOCAB_SIZE_TH = len(vocab_th)
# OUTPUT_VOCAB_SIZE_VI = len(vocab_vi)
INPUT_VOCAB_SIZE_SRC1 = len(vocab_vi) # Nguồn 1: Tiếng Việt
INPUT_VOCAB_SIZE_SRC2 = len(vocab_th) # Nguồn 2: Tiếng Thái
OUTPUT_VOCAB_SIZE_TGT = len(vocab_en) # Đích: Tiếng Anh

# model = MultiSourceTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, D_MODEL, NUM_HEADS,
#                              INPUT_VOCAB_SIZE_EN, INPUT_VOCAB_SIZE_TH, OUTPUT_VOCAB_SIZE_VI,
#                              D_FF, DROPOUT).to(device)
model = MultiSourceTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, D_MODEL, NUM_HEADS,
                             INPUT_VOCAB_SIZE_SRC1, INPUT_VOCAB_SIZE_SRC2, OUTPUT_VOCAB_SIZE_TGT,
                             D_FF, DROPOUT).to(device)

# Sử dụng AdamW thay cho Adam
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Khởi tạo GradScaler cho Mixed Precision
scaler = GradScaler()

# Khởi tạo Learning Rate Scheduler
scheduler = StepLR(optimizer, step_size=LR_SCHEDULER_STEP_SIZE, gamma=LR_SCHEDULER_GAMMA)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Mô hình có {count_parameters(model):,} tham số có thể huấn luyện.")
print(f"Batch size thực tế: {BATCH_SIZE}")
print(f"Các bước tích lũy Gradient: {GRADIENT_ACCUMULATION_STEPS}")
print(f"Batch size hiệu quả (tương đương): {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print("✅ Khởi tạo mô hình và các thành phần tối ưu hoàn tất!")

Sử dụng thiết bị: cuda


NameError: name 'vocab_vi' is not defined

In [ ]:
# Hàm save/load checkpoint giữ nguyên
def save_checkpoint(epoch, model, optimizer, scheduler, loss, filepath):
    """Lưu checkpoint."""
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': loss
    }
    torch.save(state, filepath)
    print(f"Checkpoint đã được lưu tại epoch {epoch} vào '{filepath}'")

def load_checkpoint(filepath, model, optimizer, scheduler):
    """Tải checkpoint."""
    if not os.path.exists(filepath):
        print(f"Không tìm thấy file checkpoint '{filepath}'. Bắt đầu huấn luyện từ đầu.")
        return 0, float('inf')

    checkpoint = torch.load(filepath, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint.get('loss', float('inf'))
    print(f"Đã tải checkpoint từ epoch {epoch} với loss {loss:.4f} từ '{filepath}'")
    return epoch + 1, loss


# Hàm train_epoch giữ nguyên
def train_epoch(model, dataloader, optimizer, criterion, scaler, accumulation_steps):
    """Huấn luyện một epoch với Mixed Precision và Gradient Accumulation."""
    model.train()
    total_loss = 0
    start_time = time.time()
    torch.cuda.empty_cache()

    # for i, (src_en, src_th, tgt) in enumerate(dataloader):
    #     src_en, src_th, tgt = src_en.to(device), src_th.to(device), tgt.to(device)
    #     tgt_input = tgt[:, :-1]
    #     tgt_output = tgt[:, 1:]

    #     with autocast():
    #         output = model(src_en, src_th, tgt_input)
    #         loss = criterion(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))
    #         loss = loss / accumulation_steps
    for i, (src_vi, src_th, tgt_en) in enumerate(dataloader):
        src_vi, src_th, tgt_en = src_vi.to(device), src_th.to(device), tgt_en.to(device)

        tgt_input = tgt_en[:, :-1]
        tgt_output = tgt_en[:, 1:]

        with autocast():
            # Truyền src1, src2, tgt theo đúng thứ tự của hàm forward
            output = model(src_vi, src_th, tgt_input)
            loss = criterion(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))
            loss = loss / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

        if (i + 1) % 100 == 0:
            print(f"  Batch {i+1}/{len(dataloader)}, Loss: {loss.item() * accumulation_steps:.4f}")
            torch.cuda.empty_cache()
            gc.collect()

    end_time = time.time()
    epoch_loss = total_loss / len(dataloader)
    epoch_time = end_time - start_time
    print(f"Thời gian huấn luyện epoch: {epoch_time:.2f}s")
    return epoch_loss


# Hàm translate_sentence giữ nguyên
def translate_sentence(sentence_en, sentence_th, model, max_len=50):
    model.eval()
    tokens_en = [BOS_IDX] + [vocab_en[token] for token in tokenize_en(sentence_en)] + [EOS_IDX]
    tokens_th = [BOS_IDX] + [vocab_th[token] for token in tokenize_th(sentence_th)] + [EOS_IDX]
    src_en_tensor_unbatched = torch.LongTensor(tokens_en).to(device)
    src_th_tensor_unbatched = torch.LongTensor(tokens_th).to(device)
    max_src_len = max(src_en_tensor_unbatched.size(0), src_th_tensor_unbatched.size(0))
    pad_en_needed = max_src_len - src_en_tensor_unbatched.size(0)
    pad_th_needed = max_src_len - src_th_tensor_unbatched.size(0)
    src_en_tensor = F.pad(src_en_tensor_unbatched, (0, pad_en_needed), value=PAD_IDX).unsqueeze(0)
    src_th_tensor = F.pad(src_th_tensor_unbatched, (0, pad_th_needed), value=PAD_IDX).unsqueeze(0)
    tgt_tokens = [BOS_IDX]
    for _ in range(max_len):
        tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(src_en_tensor, src_th_tensor, tgt_tensor)
        pred_token = output.argmax(2)[:, -1].item()
        tgt_tokens.append(pred_token)
        if pred_token == EOS_IDX:
            break
    tgt_words = [vocab_vi.get_itos()[i] for i in tgt_tokens]
    return " ".join(tgt_words[1:-1])
# def translate_sentence(sentence_vi, sentence_th, model, max_len=50):
#     model.eval()

#     # Tokenize các ngôn ngữ nguồn mới
#     tokens_vi = [BOS_IDX] + [vocab_vi[token] for token in tokenize_vi(sentence_vi)] + [EOS_IDX]
#     tokens_th = [BOS_IDX] + [vocab_th[token] for token in tokenize_th(sentence_th)] + [EOS_IDX]

#     src_vi_tensor = torch.LongTensor(tokens_vi).unsqueeze(0).to(device)
#     src_th_tensor = torch.LongTensor(tokens_th).unsqueeze(0).to(device)

#     # Padding đồng bộ (quan trọng cho inference)
#     max_src_len = max(src_vi_tensor.size(1), src_th_tensor.size(1))
#     src_vi_tensor = F.pad(src_vi_tensor, (0, max_src_len - src_vi_tensor.size(1)), value=PAD_IDX)
#     src_th_tensor = F.pad(src_th_tensor, (0, max_src_len - src_th_tensor.size(1)), value=PAD_IDX)

#     # Bắt đầu dịch với token <bos>
#     tgt_tokens = [BOS_IDX]
#     for _ in range(max_len):
#         tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)
#         with torch.no_grad():
#             output = model(src_vi_tensor, src_th_tensor, tgt_tensor)

#         pred_token = output.argmax(2)[:, -1].item()
#         tgt_tokens.append(pred_token)
#         if pred_token == EOS_IDX:
#             break

#     # Chuyển index về lại từ vựng Tiếng Anh
#     tgt_words = [vocab_en.get_itos()[i] for i in tgt_tokens]
#     return " ".join(tgt_words[1:-1])

# ================= SỬA LỖI TẠI ĐÂY =================
# Cập nhật hàm calculate_bleu để dọn dẹp bộ nhớ đúng cách
def calculate_bleu(dataset, model, max_pairs=100):
    targets = []
    predictions = []
    # Xử lý trường hợp dataset rỗng hoặc không có cặp nào để dịch
    if len(dataset) == 0 or max_pairs == 0:
        return 0.0

    count = 0
    for en_sent, th_sent, vi_sent in dataset:
        if count >= max_pairs:
            break
        pred_sent = translate_sentence(en_sent, th_sent, model)
        predictions.append(pred_sent)
        targets.append([vi_sent])
        count += 1

    # Xử lý trường hợp không có dự đoán nào được tạo ra
    if not predictions:
        return 0.0

    bleu = sacrebleu.corpus_bleu(predictions, targets)

    # 1. Lưu điểm số vào một biến mới
    score_to_return = bleu.score

    # 2. Xóa các biến lớn để giải phóng bộ nhớ
    del targets, predictions, bleu
    gc.collect()

    # 3. Trả về điểm số đã lưu
    return score_to_return
# ===================================================


print("✅ Các hàm chức năng đã được cập nhật với logic dọn dẹp bộ nhớ chính xác!")

✅ Các hàm chức năng đã được cập nhật với logic dọn dẹp bộ nhớ chính xác!


In [ ]:
# Chuẩn bị DataLoader
full_dataset = TranslationDataset(VI_FILE_PATH, TH_FILE_PATH, EN_FILE_PATH)
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

print(f"Kích thước tập huấn luyện: {len(train_dataset)}")
print(f"Kích thước tập validation: {len(val_dataset)}")
# Giải phóng bộ nhớ VRAM cuối epoch
torch.cuda.empty_cache()
gc.collect()

# Tải checkpoint nếu có
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'latest_checkpoint_optimized.pth') # Dùng tên file mới
start_epoch, best_val_loss = load_checkpoint(CHECKPOINT_PATH, model, optimizer, scheduler)

# Bắt đầu vòng lặp huấn luyện
for epoch in range(start_epoch, EPOCHS):
    print(f"\n--- Epoch {epoch}/{EPOCHS-1} ---")
    print(f"Learning Rate hiện tại: {scheduler.get_last_lr()[0]}")

    # Huấn luyện
    train_loss = train_epoch(model, train_dataloader, optimizer, criterion, scaler, GRADIENT_ACCUMULATION_STEPS)

    # Cập nhật learning rate scheduler
    scheduler.step()

    print(f"Epoch {epoch} | Train Loss: {train_loss:.4f}")

    # Đánh giá và tính BLEU trên tập validation
    print("⚙️ Đang đánh giá trên tập validation và tính điểm BLEU...")
    model.eval()

    # Lấy một vài câu từ validation set để dịch thử
    sample_vi, sample_th, sample_en = val_dataset[0]
    translated_sample = translate_sentence(sample_vi, sample_th, model)
    print("-" * 30)
    print(f"Câu nguồn (VI): {sample_vi}")
    print(f"Câu nguồn (TH): {sample_th}")
    print(f"Câu đích (EN) : {sample_en}")
    print(f"Câu dịch model: {translated_sample}")
    print("-" * 30)


    # Lưu checkpoint
    save_checkpoint(epoch, model, optimizer, scheduler, train_loss, CHECKPOINT_PATH)

    # Tính điểm BLEU
    #bleu_score = calculate_bleu(val_dataset, model, max_pairs=100)
    #print(f"Epoch {epoch} | BLEU score trên 200 câu validation: {bleu_score:.2f}")



    # Giải phóng bộ nhớ VRAM cuối epoch
    torch.cuda.empty_cache()

print("\n🎉 Huấn luyện hoàn tất! 🎉")
# Đánh giá và tính BLEU trên tập validation
#print("⚙️ Đang đánh giá trên tập validation và tính điểm BLEU...")
#model.eval()

# Tính điểm BLEU
#bleu_score = calculate_bleu(val_dataset, model, max_pairs=7200)
#print(f"Epoch {EPOCHS} | BLEU score trên validation: {bleu_score:.2f}")


Kích thước tập huấn luyện: 68400
Kích thước tập validation: 7600
Đã tải checkpoint từ epoch 22 với loss 1.5187 từ '/content/drive/MyDrive/TieuLuanNLP/model/multisource_vi_en/latest_checkpoint_optimized.pth'

--- Epoch 23/29 ---
Learning Rate hiện tại: 3.138105960900002e-05


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


  Batch 100/4275, Loss: 2.1113
  Batch 200/4275, Loss: 1.7864
  Batch 300/4275, Loss: 1.3930
  Batch 400/4275, Loss: 1.6814
  Batch 500/4275, Loss: 1.2656
  Batch 600/4275, Loss: 1.1284
  Batch 700/4275, Loss: 1.4779
  Batch 800/4275, Loss: 1.4848
  Batch 900/4275, Loss: 1.9809
  Batch 1000/4275, Loss: 1.8379
  Batch 1100/4275, Loss: 1.5488
  Batch 1200/4275, Loss: 2.0069
  Batch 1300/4275, Loss: 1.5792
  Batch 1400/4275, Loss: 1.3418
  Batch 1500/4275, Loss: 1.2644
  Batch 1600/4275, Loss: 1.8292
  Batch 1700/4275, Loss: 1.5958
  Batch 1800/4275, Loss: 1.8148
  Batch 1900/4275, Loss: 1.0931
  Batch 2000/4275, Loss: 1.6086
  Batch 2100/4275, Loss: 1.5655
  Batch 2200/4275, Loss: 1.3938
  Batch 2300/4275, Loss: 1.4908
  Batch 2400/4275, Loss: 1.9434
  Batch 2500/4275, Loss: 1.3582
  Batch 2600/4275, Loss: 1.4742
  Batch 2700/4275, Loss: 1.6184
  Batch 2800/4275, Loss: 1.6630
  Batch 2900/4275, Loss: 1.6249
  Batch 3000/4275, Loss: 1.5655
  Batch 3100/4275, Loss: 1.6512
  Batch 3200/4275

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:384: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


------------------------------
Câu nguồn (VI): Mã PIN có thể chưa được gửi đến bạn.
Câu nguồn (TH): - เราอาจยังไม่ได้ส่ง PIN ถึงคุณ
Câu đích (EN) : - A PIN might not have been sent to you.
Câu dịch model: - A PIN may not be sent to you .
------------------------------
Checkpoint đã được lưu tại epoch 23 vào '/content/drive/MyDrive/TieuLuanNLP/model/multisource_vi_en/latest_checkpoint_optimized.pth'

--- Epoch 24/29 ---
Learning Rate hiện tại: 2.8242953648100018e-05
  Batch 100/4275, Loss: 1.0239
  Batch 200/4275, Loss: 1.1653
  Batch 300/4275, Loss: 1.8168
  Batch 400/4275, Loss: 1.8746
  Batch 500/4275, Loss: 1.3151
  Batch 600/4275, Loss: 0.8015
  Batch 700/4275, Loss: 1.1125
  Batch 800/4275, Loss: 1.0225
  Batch 900/4275, Loss: 1.5843
  Batch 1000/4275, Loss: 1.6802
  Batch 1100/4275, Loss: 1.4955
  Batch 1200/4275, Loss: 1.4479
  Batch 1300/4275, Loss: 1.6725
  Batch 1400/4275, Loss: 1.9877
  Batch 1500/4275, Loss: 1.3663
  Batch 1600/4275, Loss: 1.7287
  Batch 1700/4275, Loss: 1.

In [ ]:
!pip install portalocker sacrebleu pythainlp underthesea torchtext==0.17.0 --quiet
# 1. Cài đặt thư viện LIME
print("⚙️ Đang cài đặt thư viện LIME...")
# Thêm phiên bản cụ thể cho LIME
!pip install lime
print("✅ Cài đặt LIME hoàn tất!")
# Fix: Install a compatible version of numpy to avoid conflicts
!pip install numpy==1.26.4 --quiet

# IMPORTANT: Restart the runtime after running this cell.
# Go to "Runtime" -> "Restart runtime".

⚙️ Đang cài đặt thư viện LIME...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=b61fbf1936afad160fbfa82499f15aca6e9784d6586d7f567e83bf57999fef66
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime
✅ Cài đặt LIME hoàn tất!


In [ ]:
# --- CELL A: LƯU MODEL CUỐI CÙNG SAU KHI HUẤN LUYỆN ---

# Kiểm tra xem mô hình đã được huấn luyện chưa
if 'model' in locals() and isinstance(model, nn.Module):
    # Định nghĩa đường dẫn và tên file cho mô hình cuối cùng
    FINAL_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'final_multisource_translator.pth')

    print(f"\n⚙️ Đang đóng gói và lưu mô hình cuối cùng...")

    # Tạo một dictionary để lưu tất cả các thành phần quan trọng
    model_save_package = {
        'epoch': EPOCHS,  # Lưu lại epoch cuối cùng đã hoàn thành
        'model_state_dict': model.state_dict(),
        'd_model': D_MODEL, 'num_heads': NUM_HEADS, 'num_encoder_layers': NUM_ENCODER_LAYERS,
        'num_decoder_layers': NUM_DECODER_LAYERS, 'd_ff': D_FF, 'dropout': DROPOUT,
        'vocab_en': vocab_en, 'vocab_th': vocab_th, 'vocab_vi': vocab_vi,
        'pad_idx': PAD_IDX,
    }

    # Thực hiện lưu file bằng lệnh torch.save() tiêu chuẩn
    torch.save(model_save_package, FINAL_MODEL_PATH)

    print(f"✅ Model cuối cùng đã được lưu thành công tại: {FINAL_MODEL_PATH}")
else:
    print("Lỗi: Không tìm thấy biến 'model'. Hãy đảm bảo bạn đã chạy các cell huấn luyện trước đó.")

Lỗi: Không tìm thấy biến 'model'. Hãy đảm bảo bạn đã chạy các cell huấn luyện trước đó.


In [ ]:
# --- CELL B: TẢI LẠI MODEL ĐỂ SỬ DỤNG (INFERENCE) ---

import math
import spacy
from pythainlp.tokenize import word_tokenize as th_word_tokenize
from underthesea import word_tokenize as vi_word_tokenize
import torch.nn.functional as F
# 1. Định nghĩa các đường dẫn và thiết bị
CHECKPOINT_DIR = '/content/drive/MyDrive/TieuLuanNLP/model/multisource_en_vi'
FINAL_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'final_multisource_translator.pth')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Tải file model đã đóng gói
print(f"⚙️ Đang tải model package từ: {FINAL_MODEL_PATH}")
model_package = torch.load(FINAL_MODEL_PATH, map_location=device, weights_only=False)

# 3. Khôi phục lại các thành phần từ package
# Lấy lại vocabularies
vocab_en = model_package['vocab_en']
vocab_th = model_package['vocab_th']
vocab_vi = model_package['vocab_vi']
PAD_IDX = model_package['pad_idx']
BOS_IDX = vocab_vi.get_stoi()['<bos>'] # Lấy lại BOS_IDX từ vocab
EOS_IDX = vocab_vi.get_stoi()['<eos>'] # Lấy lại EOS_IDX từ vocab

# Lấy lại các hyperparameters để khởi tạo đúng kiến trúc model
D_MODEL = model_package['d_model']
NUM_HEADS = model_package['num_heads']
NUM_ENCODER_LAYERS = model_package['num_encoder_layers']
NUM_DECODER_LAYERS = model_package['num_decoder_layers']
D_FF = model_package['d_ff']
DROPOUT = model_package['dropout']

INPUT_VOCAB_SIZE_EN = len(vocab_en)
INPUT_VOCAB_SIZE_TH = len(vocab_th)
OUTPUT_VOCAB_SIZE_VI = len(vocab_vi)

# 4. Khởi tạo một model rỗng và nạp trọng số vào
print("⚙️ Đang khởi tạo kiến trúc model và nạp trọng số...")
inference_model = MultiSourceTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, D_MODEL, NUM_HEADS,
                                       INPUT_VOCAB_SIZE_EN, INPUT_VOCAB_SIZE_TH, OUTPUT_VOCAB_SIZE_VI,
                                       D_FF, DROPOUT).to(device)

inference_model.load_state_dict(model_package['model_state_dict'])
inference_model.eval() # Chuyển model sang chế độ đánh giá

print("✅ Model đã sẵn sàng để dịch thuật!")

# 5. Dịch thử một câu để kiểm tra
en_test_sentence = "This is a test"
th_test_sentence = "นี่คือการทดสอบ" # "Đây là một bài kiểm tra"

# Phải định nghĩa lại các hàm tokenize
nlp_en = spacy.load('en_core_web_sm')
def tokenize_en(text): return [tok.text for tok in nlp_en.tokenizer(text)]
def tokenize_th(text): return th_word_tokenize(text)
def tokenize_vi(text): return vi_word_tokenize(text)

translated_text = translate_sentence(en_test_sentence, th_test_sentence, inference_model)

print("-" * 30)
print(f"Câu nguồn (EN): {en_test_sentence}")
print(f"Câu nguồn (TH): {th_test_sentence}")
print(f"Câu dịch model: {translated_text}")
print("-" * 30)

⚙️ Đang tải model package từ: /content/drive/MyDrive/TieuLuanNLP/model/multisource_en_vi/final_multisource_translator.pth
⚙️ Đang khởi tạo kiến trúc model và nạp trọng số...
✅ Model đã sẵn sàng để dịch thuật!
------------------------------
Câu nguồn (EN): This is a test
Câu nguồn (TH): นี่คือการทดสอบ
Câu dịch model: Đây là một bài kiểm tra
------------------------------


In [ ]:
# --- CELL ĐỂ TEST MÔ HÌNH VỚI 10 CÂU VÍ DỤ ---

# Kiểm tra xem mô hình `inference_model` đã tồn tại và sẵn sàng chưa
if 'inference_model' in locals() and isinstance(inference_model, nn.Module):

    # 1. Chuẩn bị 10 câu ví dụ
    # Lưu ý: Vì model của chúng ta là đa nguồn, nó cần cả đầu vào tiếng Anh và tiếng Thái.
    # Ở đây tôi cung cấp các cặp câu tương ứng. Nếu bạn chỉ có câu tiếng Anh,
    # bạn có thể truyền một chuỗi rỗng "" cho phần tiếng Thái.

    sample_english = [
        "Brother Albert Barnett and his wife , Sister Susan Barnett , from the West Congregation in Tuscaloosa , Alabama",
        "Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12 , 2020 .",
        "Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .",
        "Sadly , Brother Albert Barnett and his wife , Sister Susan Barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .",
        "The United States branch also reports that at least four of our brothers' homes sustained minor damage , along with two Kingdom Halls .",
        "Additionally , the storms caused major damage to a brother 's business property .",
        "Local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster .",
        "We know that our heavenly Father , Jehovah , is providing comfort to our brothers and sisters who are grieving because of this tragedy .",
        "International government agencies and officials have responded to Russia 's Supreme Court decision that criminalizes the worship of Jehovah 's Witnesses in Russia .",
        "These statements have criticized Russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity ."
    ]

    sample_thai = [
        "บราเดอร์อัลเบิร์ต บาร์เน็ตต์ และซิสเตอร์ซูซาน บาร์เน็ตต์ ภรรยาของเขา จากคริสตจักรเวสต์คองเกรสในเมืองทัสคาลูซา รัฐแอละแบมา",
        "พายุรุนแรงพัดถล่มพื้นที่ทางตอนใต้และตะวันตกกลางของสหรัฐอเมริกาเมื่อวันที่ 11 และ 12 มกราคม 2020",
        "ฝนตกหนักสองวัน ลมแรง และพายุทอร์นาโดหลายลูกสร้างความเสียหายอย่างหนักในหลายรัฐ",
        "น่าเศร้าที่บราเดอร์อัลเบิร์ต บาร์เน็ตต์ และซิสเตอร์ซูซาน บาร์เน็ตต์ ภรรยาของเขา อายุ 85 ปี และ 75 ปี ตามลำดับ เสียชีวิตเมื่อพายุทอร์นาโดพัดถล่มบ้านเคลื่อนที่ของพวกเขา",
        "สาขาในสหรัฐอเมริกายังรายงานว่าบ้านของบราเดอร์อย่างน้อยสี่หลังได้รับความเสียหายเล็กน้อย พร้อมกับหอประชุมราชอาณาจักรสองหลัง",
        "นอกจากนี้ พายุยังสร้างความเสียหายอย่างหนักต่อทรัพย์สินทางธุรกิจของบราเดอร์ท่านหนึ่ง",
        "ผู้อาวุโสในท้องถิ่นและผู้ดูแลภาคกำลังให้การสนับสนุนทั้งในทางปฏิบัติและทางจิตวิญญาณแก่ผู้ที่ได้รับผลกระทบจากภัยพิบัติครั้งนี้",
        "เรารู้ว่าพระบิดาบนสวรรค์ พระยะโฮวา ทรงประทานการปลอบโยนแก่พี่น้องชายหญิงของเราที่กำลังโศกเศร้าเสียใจจากโศกนาฏกรรมครั้งนี้",
        "หน่วยงานรัฐบาลระหว่างประเทศและเจ้าหน้าที่ได้ออกมาตอบโต้คำตัดสินของศาลฎีการัสเซียที่ทำให้การนมัสการพยานพระยะโฮวาในรัสเซียเป็นความผิดทางอาญา",
        "คำแถลงเหล่านี้วิพากษ์วิจารณ์การดำเนินการทางกฎหมายที่ไม่ยุติธรรมและรุนแรงของรัสเซียต่อกลุ่มศาสนาชนกลุ่มน้อยที่ขึ้นชื่อเรื่องการดำเนินกิจกรรมทางศาสนาอย่างสันติ",
    ]

    # Đây là các câu dịch đúng để chúng ta so sánh
    sample_vietnamese = [
        "Anh Albert Barnett và chị Susan Barnett , thuộc hội thánh West ở Tuscaloosa , Alabama",
        "Ngày 11 và 12-1-2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung Hoa Kỳ .",
        "Những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .",
        "Đáng buồn là anh Albert Barnett 85 tuổi , và vợ anh là chị Susan Barnett 75 tuổi đã thiệt mạng do một cơn lốc xoáy quét qua nhà họ .",
        "Chi nhánh Hoa Kỳ cũng cho biết có ít nhất bốn căn nhà của anh em chúng tôi và hai Phòng Nước Trời bị hư hại nhẹ .",
        "Ngoài ra , những cơn bão cũng gây hư hại lớn cho cơ sở kinh doanh của một anh em .",
        "Các trưởng lão địa phương và giám thị xung quanh đang giúp đỡ và cung cấp về vật chất và tinh thần cho các anh chị bị ảnh hưởng trong thảm hoạ này .",
        "Chúng ta tin chắc rằng Cha trên trời , Đức Giê-hô-va , đang an ủi những anh chị em của chúng ta trong cảnh đau buồn .",
        "Các cơ quan và viên chức chính phủ quốc tế đã lên tiếng trước phán quyết của Toà Tối Cao Nga về việc cấm sự thờ phượng của Nhân Chứng Giê-hô-va ở Nga .",
        "Các lời nhận xét chỉ trích nước Nga có hành động tư pháp khắc nghiệt và bất công nhắm vào một nhóm tôn giáo nhỏ được biết đến là hoạt động một cách ôn hoà ."
    ]

    print(" BẮT ĐẦU KIỂM TRA MÔ HÌNH VỚI 10 CÂU VÍ DỤ ".center(80, "="))

    # 2. Dịch từng câu và in kết quả
    for i in range(len(sample_english)):
        en_sent = sample_english[i]
        th_sent = sample_thai[i]
        vi_ref = sample_vietnamese[i]

        # Gọi hàm dịch đã được định nghĩa ở các cell trước
        model_translation = translate_sentence(en_sent, th_sent, inference_model)

        print(f"\n--- CÂU {i+1} ---")
        print(f"  > Tiếng Anh (Input 1):".ljust(25), f"{en_sent}")
        print(f"  > Tiếng Thái (Input 2):".ljust(25), f"{th_sent}")
        print(f"  > Bản dịch tham khảo:".ljust(25), f"{vi_ref}")
        print(f"  > BẢN DỊCH CỦA MODEL:".ljust(25), f"{model_translation}")

    print("=" * 80)
    print("✅ Hoàn tất kiểm tra.")

else:
    print("Lỗi: Model chưa được tải. Vui lòng chạy cell tải model (Cell B) trước khi chạy cell này.")

Lỗi: Model chưa được tải. Vui lòng chạy cell tải model (Cell B) trước khi chạy cell này.


In [ ]:
# --- CELL E: GIẢI THÍCH MODEL BẰNG KỸ THUẬT XAI LIME ---

# 2. Import các thư viện cần thiết
import lime
import lime.lime_text
from lime.lime_text import LimeTextExplainer

import sacrebleu
from IPython.display import display, HTML
from torch.nn.utils.rnn import pad_sequence
import numpy as np

# Kiểm tra xem mô hình `inference_model` đã tồn tại và sẵn sàng chưa
if 'inference_model' in locals() and isinstance(inference_model, nn.Module):

    # 2. Xây dựng một hàm "dự đoán" mới cho LIME ở chế độ Hồi quy
    # Hàm này sẽ trả về một điểm số chất lượng (thay vì xác suất) cho mỗi câu dịch.
    # Chúng ta sẽ dùng điểm SENTENCE-BLEU làm điểm chất lượng.

    # 2. Xây dựng hàm "dự đoán" mới, phỏng theo mã mẫu của bạn
    def lime_multisource_predictor(texts):
        """
        Hàm này nhận một danh sách các câu đã bị LIME xáo trộn,
        và trả về một mảng xác suất cho token ĐẦU TIÊN của câu dịch.
        Shape trả về: (số câu, kích thước bộ từ vựng đích)
        """
        en_batch_tensors, th_batch_tensors = [], []

        # Tách và token hóa từng câu
        for text in texts:
            parts = text.split(' ; ') # Phải có khoảng trắng để tránh lỗi dính chữ
            en_sent = parts[0]
            th_sent = parts[1] if len(parts) > 1 else ""

            en_tokens = [BOS_IDX] + [vocab_en[tok] for tok in tokenize_en(en_sent)] + [EOS_IDX]
            th_tokens = [BOS_IDX] + [vocab_th[tok] for tok in tokenize_th(th_sent)] + [EOS_IDX]

            en_batch_tensors.append(torch.LongTensor(en_tokens))
            th_batch_tensors.append(torch.LongTensor(th_tokens))

        # Padding đồng bộ cho cả 2 ngôn ngữ nguồn
        en_padded = pad_sequence(en_batch_tensors, batch_first=True, padding_value=PAD_IDX)
        th_padded = pad_sequence(th_batch_tensors, batch_first=True, padding_value=PAD_IDX)
        max_len = max(en_padded.size(1), th_padded.size(1))

        if en_padded.size(1) < max_len:
            en_padded = F.pad(en_padded, (0, max_len - en_padded.size(1)), 'constant', PAD_IDX)
        if th_padded.size(1) < max_len:
            th_padded = F.pad(th_padded, (0, max_len - th_padded.size(1)), 'constant', PAD_IDX)

        src_en = en_padded.to(device)
        src_th = th_padded.to(device)

        # Tạo đầu vào cho decoder: chỉ có token <BOS> để dự đoán từ đầu tiên
        batch_size = src_en.size(0)
        tgt_input = torch.LongTensor([[BOS_IDX]] * batch_size).to(device)

        # Chạy model
        with torch.no_grad():
            logits = inference_model(src_en, src_th, tgt_input)

        # Logits có shape: [batch_size, 1, vocab_size]
        probs = torch.softmax(logits, dim=-1)
        # Chuyển logits thành xác suất
        probs = probs[:, 0, :]

        return probs.cpu().numpy()
    # 3. Khởi tạo LIME Explainer ở chế độ 'regression'
    explainer = LimeTextExplainer(class_names=["vi"])

    # Lấy lại 10 câu ví dụ từ cell trước
    sample_english = [
        "Brother Albert Barnett and his wife , Sister Susan Barnett , from the West Congregation in Tuscaloosa , Alabama",
        "Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12 , 2020 .",
        "Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .",
        "Sadly , Brother Albert Barnett and his wife , Sister Susan Barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .",
        "The United States branch also reports that at least four of our brothers' homes sustained minor damage , along with two Kingdom Halls .",
        "Additionally , the storms caused major damage to a brother 's business property .",
        "Local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster .",
        "We know that our heavenly Father , Jehovah , is providing comfort to our brothers and sisters who are grieving because of this tragedy .",
        "International government agencies and officials have responded to Russia 's Supreme Court decision that criminalizes the worship of Jehovah 's Witnesses in Russia .",
        "These statements have criticized Russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity ."

    ]
    sample_thai = [
        "บราเดอร์อัลเบิร์ต บาร์เน็ตต์ และซิสเตอร์ซูซาน บาร์เน็ตต์ ภรรยาของเขา จากคริสตจักรเวสต์คองเกรสในเมืองทัสคาลูซา รัฐแอละแบมา",
        "พายุรุนแรงพัดถล่มพื้นที่ทางตอนใต้และตะวันตกกลางของสหรัฐอเมริกาเมื่อวันที่ 11 และ 12 มกราคม 2020",
        "ฝนตกหนักสองวัน ลมแรง และพายุทอร์นาโดหลายลูกสร้างความเสียหายอย่างหนักในหลายรัฐ",
        "น่าเศร้าที่บราเดอร์อัลเบิร์ต บาร์เน็ตต์ และซิสเตอร์ซูซาน บาร์เน็ตต์ ภรรยาของเขา อายุ 85 ปี และ 75 ปี ตามลำดับ เสียชีวิตเมื่อพายุทอร์นาโดพัดถล่มบ้านเคลื่อนที่ของพวกเขา",
        "สาขาในสหรัฐอเมริกายังรายงานว่าบ้านของบราเดอร์อย่างน้อยสี่หลังได้รับความเสียหายเล็กน้อย พร้อมกับหอประชุมราชอาณาจักรสองหลัง",
        "นอกจากนี้ พายุยังสร้างความเสียหายอย่างหนักต่อทรัพย์สินทางธุรกิจของบราเดอร์ท่านหนึ่ง",
        "ผู้อาวุโสในท้องถิ่นและผู้ดูแลภาคกำลังให้การสนับสนุนทั้งในทางปฏิบัติและทางจิตวิญญาณแก่ผู้ที่ได้รับผลกระทบจากภัยพิบัติครั้งนี้",
        "เรารู้ว่าพระบิดาบนสวรรค์ พระยะโฮวา ทรงประทานการปลอบโยนแก่พี่น้องชายหญิงของเราที่กำลังโศกเศร้าเสียใจจากโศกนาฏกรรมครั้งนี้",
        "หน่วยงานรัฐบาลระหว่างประเทศและเจ้าหน้าที่ได้ออกมาตอบโต้คำตัดสินของศาลฎีการัสเซียที่ทำให้การนมัสการพยานพระยะโฮวาในรัสเซียเป็นความผิดทางอาญา",
        "คำแถลงเหล่านี้วิพากษ์วิจารณ์การดำเนินการทางกฎหมายที่ไม่ยุติธรรมและรุนแรงของรัสเซียต่อกลุ่มศาสนาชนกลุ่มน้อยที่ขึ้นชื่อเรื่องการดำเนินกิจกรรมทางศาสนาอย่างสันติ",
    ]
    sample_vietnamese = [
        "Anh Albert Barnett và chị Susan Barnett , thuộc hội thánh West ở Tuscaloosa , Alabama",
        "Ngày 11 và 12-1-2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung Hoa Kỳ .",
        "Những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .",
        "Đáng buồn là anh Albert Barnett 85 tuổi , và vợ anh là chị Susan Barnett 75 tuổi đã thiệt mạng do một cơn lốc xoáy quét qua nhà họ .",
        "Chi nhánh Hoa Kỳ cũng cho biết có ít nhất bốn căn nhà của anh em chúng tôi và hai Phòng Nước Trời bị hư hại nhẹ .",
        "Ngoài ra , những cơn bão cũng gây hư hại lớn cho cơ sở kinh doanh của một anh em .",
        "Các trưởng lão địa phương và giám thị xung quanh đang giúp đỡ và cung cấp về vật chất và tinh thần cho các anh chị bị ảnh hưởng trong thảm hoạ này .",
        "Chúng ta tin chắc rằng Cha trên trời , Đức Giê-hô-va , đang an ủi những anh chị em của chúng ta trong cảnh đau buồn .",
        "Các cơ quan và viên chức chính phủ quốc tế đã lên tiếng trước phán quyết của Toà Tối Cao Nga về việc cấm sự thờ phượng của Nhân Chứng Giê-hô-va ở Nga .",
        "Các lời nhận xét chỉ trích nước Nga có hành động tư pháp khắc nghiệt và bất công nhắm vào một nhóm tôn giáo nhỏ được biết đến là hoạt động một cách ôn hoà ."
    ]



    print(" BẮT ĐẦU GIẢI THÍCH MÔ HÌNH BẰNG LIME ".center(80, "="))

    # 5. Chạy LIME cho từng câu
    for i in range(len(sample_english)):
        en_sent = sample_english[i]
        th_sent = sample_thai[i]
        vi_ref = sample_vietnamese[i]

        text_to_explain = f"{en_sent} ; {th_sent}"

        print(f"\n--- GIẢI THÍCH CHO CÂU {i+1} ---")
        print(f"Đang phân tích các từ nguồn ảnh hưởng đến điểm BLEU so với câu tham khảo: '{en_sent}'")


        # Chạy giải thích của LIME
        # num_features: số từ quan trọng nhất muốn xem
        # num_samples: số câu xáo trộn LIME tạo ra để thử nghiệm
        explanation = explainer.explain_instance(
            text_to_explain,
            lime_multisource_predictor,
            num_features=6, # Lấy 6 từ ảnh hưởng nhất
            num_samples=500 # Tăng num_samples để kết quả ổn định hơn
        )

        # Hiển thị kết quả giải thích trực quan trong notebook
        #display(HTML(explanation.as_html()))
        print(f"Explanation: {explanation.as_list()}\n")
else:
    print("Lỗi: Model chưa được tải. Vui lòng chạy cell tải model (Cell B) trước khi chạy cell này.")

===================== BẮT ĐẦU GIẢI THÍCH MÔ HÌNH BẰNG LIME =====================

--- GIẢI THÍCH CHO CÂU 1 ---
Đang phân tích các từ nguồn ảnh hưởng đến điểm BLEU so với câu tham khảo: 'Brother Albert Barnett and his wife , Sister Susan Barnett , from the West Congregation in Tuscaloosa , Alabama'


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Explanation: [('Sister', -9.458128371254788e-09), ('บาร', -6.983643577817748e-09), ('Alabama', 6.586893231817712e-09), ('กรเวสต', -6.581261933728665e-09), ('Brother', -5.165914074923474e-09), ('ภรรยาของเขา', 4.9249516469403836e-09)]


--- GIẢI THÍCH CHO CÂU 2 ---
Đang phân tích các từ nguồn ảnh hưởng đến điểm BLEU so với câu tham khảo: 'Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12 , 2020 .'


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Explanation: [('11', -5.190822749666012e-09), ('นท', -2.3475847830633787e-09), ('12', -2.341220547561406e-09), ('อว', -2.3026387707626902e-09), ('ทางตอนใต', -2.1318767306985813e-09), ('ฐอเมร', -1.965338952273433e-09)]


--- GIẢI THÍCH CHO CÂU 3 ---
Đang phân tích các từ nguồn ảnh hưởng đến điểm BLEU so với câu tham khảo: 'Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .'


...

Explanation: [('เราร', -4.7995118168464256e-09), ('We', -3.4862006587222655e-09), ('ดาบนสวรรค', -3.1818698508450587e-09), ('providing', 2.6072726160661064e-09), ('are', -2.4018433614400314e-09), ('พระยะโฮวา', -2.0913350460722445e-09)]


--- GIẢI THÍCH CHO CÂU 9 ---
Đang phân tích các từ nguồn ảnh hưởng đến điểm BLEU so với câu tham khảo: 'International government agencies and officials have responded to Russia 's Supreme Court decision that criminalizes the worship of Jehovah 's Witnesses in Russia .'


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Explanation: [('decision', -3.2014751213721923e-09), ('วยงานร', -3.164850234714558e-09), ('างประเทศและเจ', -2.8643337867162418e-09), ('หน', -2.030721277986848e-09), ('that', -1.8602313934959648e-09), ('s', 1.5595094341127537e-09)]


--- GIẢI THÍCH CHO CÂU 10 ---
Đang phân tích các từ nguồn ảnh hưởng đến điểm BLEU so với câu tham khảo: 'These statements have criticized Russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity .'


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Explanation: [('These', -1.071552542889616e-08), ('นการทางกฎหมายท', -3.2609704978947955e-09), ('have', -2.3760834263332615e-09), ('จกรรมทางศาสนาอย', -2.2072951349011548e-09), ('าน', -2.1120428582240097e-09), ('known', -1.595341121168384e-09)]



# **III - Mô hình với tăng cường dữ liệu**

**Bước 1: Xử lí dữ liệu**

In [ ]:
import re

def clean_text(text):
    # Bỏ dấu ngoặc, ký hiệu đặc biệt (giữ lại dấu chấm, dấu hỏi, dấu phẩy, chữ)
    text = re.sub(r'[\(\)\[\]\{\}"\':=]', '', text)

    # Bỏ biểu cảm kiểu =))), :))), :D
    text = re.sub(r'[=:;][)D]+', '', text)

    # Bỏ dấu ? nếu đứng sau biểu cảm
    text = re.sub(r'[\)=]+[\?]+', '', text)

    # Rút gọn nhiều dấu ? về 1 dấu ?
    text = re.sub(r'\?{2,}', '?', text)

    # Rút gọn nhiều dấu chấm (...) về 1 chấm
    text = re.sub(r'\.{2,}', '.', text)

    # Bỏ các tag HTML, markdown nếu có
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\*+', '', text)

    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    return text if re.search(r'\w', text) else None
input_path = "../data_augment/train.en"
output_path = ""

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        cleaned = clean_text(line)
        if cleaned:
            outfile.write(cleaned + "\n")


**Bước 2: Sử dụng GG Translator để dịch ngược**

In [ ]:
from deep_translator import GoogleTranslator
import os
import yaml
import time
from tqdm import tqdm
from multiprocessing import Pool, cpu_count, Manager, Lock
import sys
sys.stdout.reconfigure(encoding='utf-8')

CACHE_FILE = "data_augment/cache_translate.txt"
NUM_PROCESSES = 12

def load_cache():
    if not os.path.exists(CACHE_FILE):
        return {}
    with open(CACHE_FILE, "r", encoding="utf-8") as f:
        lines = [line.strip().split("|||") for line in f if "|||" in line]
    return {src: tgt for src, tgt in lines}

def update_cache(cache):
    with open(CACHE_FILE, "a", encoding="utf-8") as f:
        for src, tgt in cache.items():
            f.write(f"{src}|||{tgt}\n")

def translate_line(args):
    line, src_lang, dest_lang, cache_shared, output_path, lock = args

    if line in cache_shared:
        translated = cache_shared[line]
    else:
        try:
            translated = GoogleTranslator(source=src_lang, target=dest_lang).translate(line)
            # Kiểm tra nếu không đổi thì đánh dấu là lỗi
            if translated.strip() == line.strip():
                print(f" Không dịch được: {line}")
        except Exception:
            translated = line
        cache_shared[line] = translated

        with lock:
            with open(output_path, "a", encoding="utf-8") as f_out:
                f_out.write(translated + "\n")

    return line, translated

def run_translation_block(config_block, shared_cache):
    src_lang = config_block["src_lang"]
    dest_lang = config_block["dest_lang"]
    input_file = config_block["input_data"]
    output_file = config_block["output_data"]

    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(input_file, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    already_done = 0
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            already_done = len([line for line in f if line.strip()])
        print(f"Tiếp tục từ dòng {already_done}/{len(lines)}")

    lines = lines[already_done:]

    if not lines:
        print(f" Đã dịch xong: {output_file}")
        return

    print(f"\nBắt đầu dịch: {src_lang} → {dest_lang} ({len(lines)} dòng)")
    start_time = time.time()

    manager = Manager()
    lock = manager.Lock()

    args = [(line, src_lang, dest_lang, shared_cache, output_file, lock) for line in lines]

    with Pool(processes=min(cpu_count(), NUM_PROCESSES)) as pool:
        results = list(tqdm(pool.imap(translate_line, args), total=len(args)))

    update_cache(dict(results))

    elapsed_time = time.time() - start_time
    print(f "Dịch xong: {output_file} ({int(elapsed_time // 60)} phút {int(elapsed_time % 60)} giây)")

def run_backtranslation(config_path):
    with open(config_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)["back_translation"]

    manager = Manager()
    shared_cache = manager.dict(load_cache())

    start = time.time()
    # run_translation_block(config["vi_to_en"], shared_cache)
    run_translation_block(config["en_to_vi"], shared_cache)
    print(f"\nTổng thời gian: {int((time.time() - start) // 60)} phút")

if __name__ == "__main__":
    run_backtranslation("../model/data_augment/config/config.yaml")



**Bước 3: Xây dựng bảng cắt tỉa cụm từ**

In [ ]:
!git clone https://github.com/clab/fast_align.git
%cd fast_align
!mkdir build
%cd build
!cmake ..
!make


In [ ]:
# Đường dẫn file
en_file = "../data_augment/train.en"
vi_file = "../data_augment/train.vi"
output_file = "../data_augment/aligned.txt"

# Đọc dữ liệu
with open(en_file, "r", encoding="utf-8") as f_en, \
     open(vi_file, "r", encoding="utf-8") as f_vi:

    en_lines = [line.strip() for line in f_en]
    vi_lines = [line.strip() for line in f_vi]

# Kiểm tra độ dài
assert len(vi_lines) == len(en_lines)

# Gộp và ghi ra file
with open(output_file, "w", encoding="utf-8") as f_out:
    for en, vi in zip(en_lines, vi_lines):
        f_out.write(f"{en} ||| {vi}\n")

print(f"Đã tạo file {output_file} với {len(en_lines)} dòng.")


In [ ]:
!./fast_align -i ../data_augment/aligned.txt -d -o -v > ../data_augment/forward.align

In [ ]:
import os
import yaml
import json
from collections import defaultdict, Counter

EN_STOPWORDS = {
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves",
    "you", "your", "yours", "yourself", "yourselves",
    "he", "him", "his", "himself", "she", "her", "hers", "herself",
    "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those",
    "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having",
    "do", "does", "did", "doing",
    "a", "an", "the",
    "and", "but", "if", "or", "because", "as", "until", "while",
    "of", "at", "by", "for", "with", "about", "against", "between",
    "into", "through", "during", "before", "after", "above", "below",
    "to", "from", "up", "down", "in", "out", "on", "off", "over", "under",
    "again", "further", "then", "once", "here", "there", "when", "where",
    "why", "how", "all", "any", "both", "each", "few", "more", "most",
    "other", "some", "such", "no", "nor", "not", "only", "own", "same",
    "so", "than", "too", "very",
    "s", "t", "can", "will", "just", "don", "should", "now"
}

def clean_phrase(phrase):
    return phrase.strip().strip('"“”‘’`\'.,:;!?')

def load_config(path="../model/data_augment/config/config.yaml"):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)["pruning"]

def load_alignments(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip().split() for line in f.readlines()]

def extract_phrase_pairs(en_data, vi_data, align_file, output_file, max_ngram=3):
    def extract_aligned_phrases(en_words, vi_words, alignments, max_n=3):
        phrase_table = []
        align_dict = defaultdict(list)
        for a in alignments:
            try:
                e_idx, v_idx = map(int, a.split('-'))
                align_dict[e_idx].append(v_idx)
            except:
                continue

        for e_start in range(len(en_words)):
            for e_len in range(1, max_n + 1):
                e_end = e_start + e_len
                if e_end > len(en_words):
                    continue

                e_indices = list(range(e_start, e_end))
                v_indices = []
                for ei in e_indices:
                    v_indices.extend(align_dict.get(ei, []))
                if not v_indices:
                    continue

                v_start = min(v_indices)
                v_end = max(v_indices) + 1
                if v_end > len(vi_words):
                    continue

                en_phrase = clean_phrase(" ".join(en_words[e_start:e_end]))
                vi_phrase = clean_phrase(" ".join(vi_words[v_start:v_end]))
                if en_phrase and vi_phrase:
                    phrase_table.append((en_phrase, vi_phrase))
        return phrase_table

    with open(en_data, 'r', encoding='utf-8') as f_en, \
         open(vi_data, 'r', encoding='utf-8') as f_vi:
        en_lines = [line.strip().split() for line in f_en]
        vi_lines = [line.strip().split() for line in f_vi]

    alignments = load_alignments(align_file)
    phrase_table = defaultdict(lambda: defaultdict(int))

    for idx, (en_words, vi_words, align_line) in enumerate(zip(en_lines, vi_lines, alignments)):
        phrase_pairs = extract_aligned_phrases(en_words, vi_words, align_line, max_ngram)
        for en_phrase, vi_phrase in phrase_pairs:
            phrase_table[en_phrase][vi_phrase] += 1

    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, "w", encoding="utf-8") as f_out:
        for en, vi_dict in phrase_table.items():
            for vi, count in vi_dict.items():
                f_out.write(f"{en} ||| {vi} ||| {count}\n")

    print(f"Đã trích phrase-pair 1–{max_ngram} từ → {output_file}")
    return phrase_table

def prune_and_save_dict(phrase_table, phrase_dict_path, threshold):
    phrase_dict = {}
    for en_phrase, vi_counter in phrase_table.items():
        if en_phrase.lower() in EN_STOPWORDS:
            continue

        filtered_vi = [(vi, count) for vi, count in vi_counter.items() if count >= threshold]
        if filtered_vi:
            top_vi = max(filtered_vi, key=lambda x: x[1])[0]
            en_clean = clean_phrase(en_phrase)
            vi_clean = clean_phrase(top_vi)
            if en_clean and vi_clean:
                phrase_dict[en_clean] = vi_clean

    with open(phrase_dict_path, 'w', encoding='utf-8') as f_out:
        json.dump(phrase_dict, f_out, ensure_ascii=False, indent=2)

    print(f"Đã lưu phrase dictionary → {phrase_dict_path}")

def run_pipeline():
    config = load_config()

    print("Trích cụm từ từ alignments...")
    phrase_table = extract_phrase_pairs(
        en_data=config["en_data"],
        vi_data=config["vi_data"],
        align_file=config["align_file"],
        output_file=config["phrase_table"],
        max_ngram=config.get("max_ngram", 3)
    )

    print("Cắt tỉa và xuất dictionary...")
    prune_and_save_dict(
        phrase_table=phrase_table,
        phrase_dict_path=config["phrase_dict"],
        threshold=config["threshold"]
    )

    print("Đã xong")

if __name__ == "__main__":
    run_pipeline()


**Bước 4: Gộp tất cả dữ liệu lại cho việc huấn luyện mô hình Transformer**

In [ ]:
import yaml
import json
import random
import os
import sys

# Đảm bảo stdout ghi ra UTF-8
sys.stdout.reconfigure(encoding='utf-8')

def load_config(config_path):
    with open(config_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

def load_phrase_dict(phrase_dict_file):
    with open(phrase_dict_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def combine_data(raw_en, raw_vi, backtrans_en, backtrans_vi, out_en_file, out_vi_file):
    os.makedirs(os.path.dirname(out_en_file), exist_ok=True)

    en_lines = []
    vi_lines = []

    with open(raw_en, 'r', encoding='utf-8') as f:
        en_lines.extend([line.strip() for line in f if line.strip()])
    with open(raw_vi, 'r', encoding='utf-8') as f:
        vi_lines.extend([line.strip() for line in f if line.strip()])

    with open(backtrans_en, 'r', encoding='utf-8') as f:
        en_lines.extend([line.strip() for line in f if line.strip()])
    with open(backtrans_vi, 'r', encoding='utf-8') as f:
        vi_lines.extend([line.strip() for line in f if line.strip()])

    with open(out_en_file, 'w', encoding='utf-8') as f_en:
        for line in en_lines:
            f_en.write(line + '\n')
    with open(out_vi_file, 'w', encoding='utf-8') as f_vi:
        for line in vi_lines:
            f_vi.write(line + '\n')

    return en_lines, vi_lines

def augment_from_phrase_dict(phrase_dict, max_samples=10000):
    augmented_en = []
    augmented_vi = []

    items = list(phrase_dict.items())
    random.shuffle(items)

    for en_phrase, vi_phrase in items[:max_samples]:
        en_phrase = en_phrase.strip()
        vi_phrase = vi_phrase.strip()
        if en_phrase and vi_phrase:
            augmented_en.append(en_phrase)
            augmented_vi.append(vi_phrase)

    return augmented_en, augmented_vi

def save_augmented_data(en_lines, vi_lines, out_en_file, out_vi_file):
    with open(out_en_file, 'w', encoding='utf-8') as f_en:
        for line in en_lines:
            f_en.write(line + '\n')

    with open(out_vi_file, 'w', encoding='utf-8') as f_vi:
        for line in vi_lines:
            f_vi.write(line + '\n')

if __name__ == "__main__":
    config = load_config("../model/data_augment/config/config.yaml")

    # Bước 1: Gộp dữ liệu gốc + back-translation
    en_lines, vi_lines = combine_data(
        config['nmt']['en_data'],
        config['nmt']['vi_data'],
        config['back_translation']['vi_to_en']['output_data'],
        config['back_translation']['en_to_vi']['output_data'],
        "../data_augment/train_combined.en",
        "../data_augment/train_combined.vi"
    )

    # Bước 2: Làm giàu từ phrase_dict
    phrase_dict = load_phrase_dict(config['pruning']['phrase_dict'])
    aug_en, aug_vi = augment_from_phrase_dict(phrase_dict, max_samples=10000)

    # Bước 3: Gộp toàn bộ vào tập huấn luyện mở rộng
    total_en = en_lines + aug_en
    total_vi = vi_lines + aug_vi

    save_augmented_data(
        total_en,
        total_vi,
        "../data_augment/train_augmented.en",
        "../data_augment/train_augmented.vi"
    )

    print(f"Đã lưu train_augmented.en & train_augmented.vi ({len(total_en)} cặp)")


**Bước 5: Xây dựng mô hình Transformer và tiến hành huấn luyện, đánh giá**

In [ ]:
import torch
import torch.nn as nn
from transformers import MarianMTModel, MarianTokenizer

class NMTModel(nn.Module):
    def __init__(self, model_name="Helsinki-NLP/opus-mt-en-vi"):
        super(NMTModel, self).__init__()
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.model = MarianMTModel.from_pretrained(model_name)

    def forward(self, src_ids, src_mask, tgt_ids, tgt_mask):
        outputs = self.model(input_ids=src_ids, attention_mask=src_mask, decoder_input_ids=tgt_ids, decoder_attention_mask=tgt_mask)
        return outputs.logits

    def translate(self, text, max_length=128):
        self.model.eval()
        device = next(self.model.parameters()).device
        inputs = self.tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=max_length,
                num_beams=4,
                early_stopping=True,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
  import yaml
  import os
  import torch
  from torch.utils.data import Dataset, DataLoader, random_split
  from transformers import MarianMTModel, MarianTokenizer
  from tqdm import tqdm
  import sacrebleu
  import matplotlib.pyplot as plt


  # Dataset NMT
  class MarianDataset(Dataset):
      def __init__(self, src_path, tgt_path, tokenizer, max_length=128):
          self.src_lines = self._load_file(src_path)
          self.tgt_lines = self._load_file(tgt_path)
          assert len(self.src_lines) == len(self.tgt_lines), "[!] Số dòng không khớp giữa EN và VI"
          self.tokenizer = tokenizer
          self.max_length = max_length

      def _load_file(self, path):
          with open(path, "r", encoding="utf-8") as f:
              return [line.strip() for line in f if line.strip()]

      def __len__(self):
          return len(self.src_lines)

      def __getitem__(self, idx):
          src = self.src_lines[idx]
          tgt = self.tgt_lines[idx]

          inputs = self.tokenizer(
              src,
              truncation=True,
              padding='max_length',
              max_length=self.max_length,
              return_tensors="pt"
          )
          labels = self.tokenizer(
              tgt,
              truncation=True,
              padding='max_length',
              max_length=self.max_length,
              return_tensors="pt"
          ).input_ids

          return {
              "input_ids": inputs.input_ids.squeeze(),
              "attention_mask": inputs.attention_mask.squeeze(),
              "labels": labels.squeeze()
          }

  # Config
  def load_config(config_path):
      with open(config_path, 'r', encoding='utf-8') as f:
          return yaml.safe_load(f)


  # Checkpoint
  def save_checkpoint(model, optimizer, epoch, model_dir):
      path = os.path.join(model_dir, "checkpoint_last.pt")
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict()
      }, path)

  def load_checkpoint(model, optimizer, model_dir, device):
      path = os.path.join(model_dir, "checkpoint_last.pt")
      if os.path.isfile(path):
          checkpoint = torch.load(path, map_location=device)
          model.load_state_dict(checkpoint['model_state_dict'])
          optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
          print(f"Resumed from checkpoint at epoch {checkpoint['epoch']}")
          return checkpoint['epoch'] + 1
      return 0


  # Evaluate BLEU
  def evaluate_bleu(model, tokenizer, dataloader, device, max_batches=20):
      model.eval()
      refs, hyps = [], []
      with torch.no_grad():
          for i, batch in enumerate(dataloader):
              if i >= max_batches:
                  break
              input_ids = batch["input_ids"].to(device)
              attention_mask = batch["attention_mask"].to(device)
              labels = batch["labels"]

              outputs = model.generate(
                  input_ids=input_ids,
                  attention_mask=attention_mask,
                  max_length=128,
                  num_beams=1
              )
              decoded_preds = [tokenizer.decode(g, skip_special_tokens=True) for g in outputs]
              decoded_labels = [tokenizer.decode(l, skip_special_tokens=True) for l in labels]

              hyps.extend(decoded_preds)
              refs.extend(decoded_labels)

      bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
      model.train()
      return bleu

  # Plot Loss & BLEU
  def plot_metrics(log_path="logs/train.log", save_path="logs/metrics.png"):
      epochs, losses, bleus = [], [], []
      with open(log_path, "r", encoding="utf-8") as f:
          for line in f:
              parts = line.strip().split(",")
              epoch = int(parts[0].split()[1])
              loss = float(parts[1].split(":")[1])
              bleu = float(parts[2].split(":")[1])
              epochs.append(epoch)
              losses.append(loss)
              bleus.append(bleu)

      plt.figure(figsize=(10,5))

      plt.subplot(1,2,1)
      plt.plot(epochs, losses, marker='o', color='red')
      plt.title("Training Loss")
      plt.xlabel("Epoch")
      plt.ylabel("Loss")

      plt.subplot(1,2,2)
    plt.plot(epochs, bleus, marker='o', color='blue')
    plt.title("Validation BLEU")
    plt.xlabel("Epoch")
    plt.ylabel("BLEU")

    plt.tight_layout()
    plt.savefig(save_path)
    print(f"Saved training curves to {save_path}")

# Train NMT
def train_nmt(config_path):
    config = load_config(config_path)['nmt']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = MarianTokenizer.from_pretrained(config['model_name'])
    model = MarianMTModel.from_pretrained(config['model_name']).to(device)

    dataset = MarianDataset(
        src_path=config['train_en'],
        tgt_path=config['train_vi'],
        tokenizer=tokenizer,
        max_length=config.get('max_len', 128)
    )

    train_size = int(0.8 * len(dataset))
    val_size = min(200, int(0.1 * len(dataset)))
    test_size = len(dataset) - train_size - val_size
    train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

    train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config['batch_size'])
    test_loader = DataLoader(test_set, batch_size=config['batch_size'])

    optimizer = torch.optim.AdamW(model.parameters(), lr=float(config['learning_rate']))

    os.makedirs(config['model_dir'], exist_ok=True)
    os.makedirs("logs", exist_ok=True)

    start_epoch = load_checkpoint(model, optimizer, config['model_dir'], device)

    print(f"Training on {device} (starting from epoch {start_epoch})")
    for epoch in range(start_epoch, config['epochs']):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)

        val_bleu = evaluate_bleu(model, tokenizer, val_loader, device, max_batches=20)

        torch.save(model.state_dict(), os.path.join(config['model_dir'], f"nmt_epoch_{epoch}.pt"))
        save_checkpoint(model, optimizer, epoch, config['model_dir'])

        with open("logs/train.log", 'a', encoding='utf-8') as f:
            f.write(f"Epoch {epoch}, Loss:{avg_loss:.4f}, ValBLEU:{val_bleu:.2f}\n")

        print(f"Epoch {epoch} — Avg Loss: {avg_loss:.4f}, Val BLEU: {val_bleu:.2f}")

    test_bleu = evaluate_bleu(model, tokenizer, test_loader, device, max_batches=50)
    print(f"\nFinal Test BLEU: {test_bleu:.2f}")

    plot_metrics("../data_augment/logs/train.log", "../data_augment/metrics.png")

if __name__ == "__main__":
    train_nmt("../model/data_augment/config/config.yaml")


# **IV - Các hàm đánh giá và xử lý chung**

**1 - Xây dựng mô hình sửa lỗi ngữ pháp và teencode**

In [ ]:
import os
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)
from transformers.trainer_utils import get_last_checkpoint

# === Cấu hình ===
PRETRAINED_MODEL = "VietAI/vit5-base"
CHECKPOINT_DIR = "../model/data_augment/checkpoints/gec"
LOG_DIR = "../model/data_augment/gec"
SRC_PATH = "../data_augment/train.src"
TGT_PATH = "../data_augment/train.tgt"

os.makedirs(LOG_DIR, exist_ok=True)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# === Tải dữ liệu gốc ===
with open(SRC_PATH, encoding="utf-8") as f:
    src_lines = [line.strip() for line in f if line.strip()]
with open(TGT_PATH, encoding="utf-8") as f:
    tgt_lines = [line.strip() for line in f if line.strip()]
assert len(src_lines) == len(tgt_lines), "Số dòng không khớp!"

# === Chia train/val ===
train_src, val_src, train_tgt, val_tgt = train_test_split(
    src_lines, tgt_lines, test_size=0.05, random_state=42
)
train_data = Dataset.from_dict({"input": train_src, "target": train_tgt})
val_data = Dataset.from_dict({"input": val_src, "target": val_tgt})

# === Kiểm tra checkpoint gần nhất ===
last_ckpt = get_last_checkpoint(CHECKPOINT_DIR)
if last_ckpt:
    print(f"Đang tiếp tục huấn luyện từ checkpoint: {last_ckpt}")
    # Xoá file rng_state.pth để tránh lỗi UnpicklingError (PyTorch 2.6+)
    rng_file = os.path.join(last_ckpt, "rng_state.pth")
    if os.path.exists(rng_file):
        os.remove(rng_file)
    tokenizer = AutoTokenizer.from_pretrained(last_ckpt)
    model = AutoModelForSeq2SeqLM.from_pretrained(last_ckpt)
else:
    print("Bắt đầu huấn luyện mới từ mô hình gốc...")
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Thiết bị:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# === Tiền xử lý dữ liệu ===
max_len = 128
def preprocess(example):
    inputs = tokenizer("gec: " + example["input"], truncation=True, padding="max_length", max_length=max_len)
    targets = tokenizer(example["target"], truncation=True, padding="max_length", max_length=max_len)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_train = train_data.map(preprocess, batched=False)
tokenized_val = val_data.map(preprocess, batched=False)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# === Cấu hình huấn luyện ===
training_args = Seq2SeqTrainingArguments(
    output_dir=CHECKPOINT_DIR,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    logging_dir=LOG_DIR,
    logging_steps=50,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=1,
    optim="adafactor",
    fp16=torch.cuda.is_available(),
    overwrite_output_dir=False,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# === Huấn luyện ===
if __name__ == "__main__":
    trainer.train(resume_from_checkpoint=last_ckpt if last_ckpt else None)
    model.save_pretrained(CHECKPOINT_DIR)
    tokenizer.save_pretrained(CHECKPOINT_DIR)
    print(f"Đã lưu mô hình vào: {CHECKPOINT_DIR}")


**2 - Sử dụng mô hình sửa lỗi ngữ pháp để xử lý dữ liệu đầu vào**

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

# === Cấu hình ===
CHECKPOINT_DIR = "../model/data_augment/checkpoints/gec/checkpoint-16238"
INPUT_FILE = "../data_augment/train.vi"
OUTPUT_FILE = "../data_augment/train.corrected.vi"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 128
MAX_LEN = 128
PREFIX = "gec: "

# === Tải model và tokenizer ===
print(" Đang tải model và tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT_DIR)

if torch.cuda.is_available():
    model = model.half()
model.to(DEVICE)
model.eval()
print(f"Model loaded on: {DEVICE}")

# === Đọc dữ liệu cần sửa lỗi ===
with open(INPUT_FILE, encoding="utf-8") as f:
    all_lines = [line.strip() for line in f if line.strip()]

# === Nếu đã có kết quả, tiếp tục từ chỗ dừng ===
start_idx = 0
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, encoding="utf-8") as f:
        done_lines = sum(1 for _ in f)
        start_idx = done_lines
        print(f" Đang tiếp tục từ dòng {start_idx}/{len(all_lines)}")

# === Mở file ghi kết quả ===
with open(OUTPUT_FILE, "a", encoding="utf-8") as fout:
    for i in tqdm(range(start_idx, len(all_lines), BATCH_SIZE), desc="Sửa lỗi"):
        batch = all_lines[i:i + BATCH_SIZE]
        inputs = tokenizer(
            [PREFIX + x for x in batch],
            return_tensors="pt",
            padding="longest",
            truncation=True,
            max_length=MAX_LEN
        ).to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=MAX_LEN,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        fout.write("\n".join(decoded) + "\n")

print(f"\n Hoàn tất sửa lỗi. Kết quả lưu tại: {OUTPUT_FILE}")


**3 - Đánh giá BLEU Score**

In [ ]:
!pip install sacrebleu pandas nltk
import sentencepiece as spm
import json
import gc
import nltk
from nltk import word_tokenize, pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
# Hàm đánh giá (sửa để không dùng generate)
def evaluate_model_multisource(model, test_data, model_name):
    model.eval()
    predictions = []
    references = test_data["vi"].tolist()
    batch_size = 32  # Giảm từ 32 xuống 16 để tiết kiệm VRAM

    # Tải từ vựng POS và SentencePiece
    drive_path = '/content/drive/My Drive/TieuLuanNLP/multi_source/'
    with open(os.path.join('/content/drive/My Drive/TieuLuanNLP/model/multisource_en_vi/', 'pos_vocab.json'), 'r', encoding='utf-8') as f:
      pos_vocab = json.load(f)

    src_sp = spm.SentencePieceProcessor()
    src_sp.load(os.path.join(drive_path, 'src_sp.model'))
    tgt_sp = spm.SentencePieceProcessor()
    tgt_sp.load(os.path.join(drive_path, 'tgt_sp.model'))

    for i in range(0, len(test_data), batch_size):
        batch = test_data.iloc[i:i+batch_size]
        src_texts = batch["en"].tolist()

        # Token hóa và tạo POS tags
        src_ids_list, ling_ids_list = [], []
        for text in src_texts:
            tokens = word_tokenize(text)[:128]
            src_ids = [src_sp.piece_to_id(token) for token in tokens]
            pos_tags = [pos_vocab.get(tag, 0) for _, tag in pos_tag(tokens)]
            src_ids_list.append(src_ids)
            ling_ids_list.append(pos_tags)

        # Padding
        src_ids = torch.tensor([ids + [0] * (128 - len(ids)) if len(ids) < 128 else ids[:128]
                               for ids in src_ids_list], dtype=torch.long).to(device)
        ling_ids = torch.tensor([ids + [0] * (128 - len(ids)) if len(ids) < 128 else ids[:128]
                                for ids in ling_ids_list], dtype=torch.long).to(device)

        # Khởi tạo tgt_ids với <s>
        tgt_ids = torch.tensor([[tgt_sp.piece_to_id('<s>')] for _ in range(len(src_texts))],
                              dtype=torch.long, device=device)

        # Suy luận
        with torch.no_grad():
            for _ in range(50):  # max_length=50
                output = model(src_ids, ling_ids, tgt_ids)
                next_token = output[:, -1, :].argmax(dim=-1)
                tgt_ids = torch.cat((tgt_ids, next_token.unsqueeze(1)), dim=1)
                if (next_token == tgt_sp.piece_to_id('</s>')).all():
                    break

        # Giải mã
        batch_predictions = [tgt_sp.decode_ids(tgt.tolist()) for tgt in tgt_ids.cpu()]
        predictions.extend(batch_predictions)

        # Giải phóng bộ nhớ
        del src_ids, ling_ids, tgt_ids, output
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    bleu = sacrebleu.corpus_bleu(predictions, [references]).score
    print(f"{model_name} BLEU Score: {bleu}")

    return predictions

def evaluate_model(model, test_data, model_name):
    model.eval()
    predictions = []
    references = test_data["vi"].tolist()
    for i in range(0, len(test_data), 32):
        batch = test_data.iloc[i:i+32]
        inputs = tokenizer(batch["en"].tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
        del inputs, outputs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()  # Clear GPU memory
        gc.collect()
    bleu = sacrebleu.corpus_bleu(predictions, [references]).score
    print(f"{model_name} BLEU Score: {bleu}")
    return predictions
from transformers import MarianMTModel

# Tải mô hình đã huấn luyện
#model_basic = MarianMTModel.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/basic_en_vn").to(device)
model_multi = MultiSourceTransformer.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/multisource_en_vi").to(device)
#model_aug = MarianMTModel.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/data_augment_en_vi/checkpoints/nmt").to(device)

# Đánh giá
#preds_basic = evaluate_model(model_basic, test_phomt, "Transformer cơ bản")
#preds_aug = evaluate_model(model_aug, test_phomt, "Mô hình tăng cường")
preds_multi = evaluate_model_multisource(model_multi, test_phomt, "Mô hình đa nguồn")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


AttributeError: 'MultiSourceTransformer' object has no attribute 'src_tokenizer'

Transformer cơ bản BLEU Score: 27.29405794636997

Mô hình tăng cường BLEU Score: 23.211848832715262

Mô hình đa nguồn BLEU score: 54.73

**4 - Sử dụng XAI LIME để giải thích các mô hình**

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from lime.lime_text import LimeTextExplainer
from google.colab import drive
drive.mount('/content/drive')

# Load mô hình
#model =  MarianMTModel.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/basic_en_vn")
model =  MarianMTModel.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/data_augment_en_vi/checkpoints/nmt")
#model = MultiSourceTransformer.from_pretrained("/content/drive/My Drive/TieuLuanNLP/model/multisource_en_vi")

# LIME giải thích
explainer = LimeTextExplainer(class_names=["vi"])

def predict_proba_basic(texts):
    if isinstance(texts, str):
        texts = [texts]
    elif not isinstance(texts, list):
        raise ValueError("Đầu vào cho predict_proba_basic phải là str hoặc list[str]")
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    # Tạo decoder_input_ids thủ công (dùng token bắt đầu <s> cho decoder)
    decoder_input_ids = torch.ones((inputs["input_ids"].size(0), 1), dtype=torch.long, device="cpu") * tokenizer.convert_tokens_to_ids("<s>")
    with torch.no_grad():
        # Gọi mô hình với decoder_input_ids rõ ràng, tắt decoder_inputs_embeds
        outputs = model(
            **inputs,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,  # Tắt cache để giảm bộ nhớ
            output_hidden_states=False,
            output_attentions=False
        )
        logits = outputs.logits  # Logits của các token dự đoán
        # Chuyển logits thành xác suất
        probs = torch.softmax(logits, dim=-1)  # Shape: [batch_size, seq_len, vocab_size]
        probs = probs[:, 0, :]  # Lấy xác suất của token đầu tiên cho mỗi mẫu
    # Trả về mảng xác suất (shape: [num_samples, vocab_size])
    return probs.cpu().numpy()

for i in range(10):
    text = test_phomt.iloc[i]["en"]
    if pd.isna(text) or not isinstance(text, str):
        print(f"Lỗi: Dữ liệu tại dòng {i} không hợp lệ: {text}")
        continue
    exp = explainer.explain_instance(text, predict_proba_basic, num_features=6, num_samples=500)
    #exp = explainer.explain_instance(text, predict_proba_mutisource, num_features=6, num_samples=500)
    print(f"Text: {text}")
    print(f"Explanation: {exp.as_list()}\n")

ImportError: cannot import name 'MarianMTModel' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

**1- bảng dịch transformer cơ bản**

```
Text: brother albert barnett and his wife , sister susan barnett , from the west congregation in tuscaloosa , alabama

Explanation: [(np.str_('his'), -2.0996182684032016e-06), (np.str_('susan'), -1.4826946145904694e-06), (np.str_('brother'), 8.91575255720878e-07), (np.str_('barnett'), 7.523639917550182e-07), (np.str_('albert'), 7.229438778146044e-07), (np.str_('alabama'), 6.242862921000612e-07)]

Text: severe storms ripped through parts of the southern and midwestern united states on january 11 and 12 , 2020 .

Explanation: [(np.str_('ripped'), -1.4482198924350898e-06), (np.str_('storms'), -1.1239242709494548e-06), (np.str_('on'), -9.466474156549832e-07), (np.str_('severe'), -5.531836322864629e-07), (np.str_('2020'), -4.989570001045183e-07), (np.str_('of'), -4.3672736907293595e-07)]

Text: two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .

Explanation: [(np.str_('rain'), -1.3265755932110205e-06), (np.str_('days'), -5.167963272566775e-07), (np.str_('of'), 3.542482333299923e-07), (np.str_('across'), -3.0864738658966563e-07), (np.str_('major'), 3.07773549879045e-07), (np.str_('caused'), 1.3952172389578821e-07)]

Text: sadly , brother albert barnett and his wife , sister susan barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .

Explanation: [(np.str_('sadly'), -5.516835676895757e-06), (np.str_('and'), -7.265597596709589e-07), (np.str_('mobile'), 6.022623230916172e-07), (np.str_('his'), -4.2567762722337795e-07), (np.str_('old'), 3.6071062564500606e-07), (np.str_('barnett'), 2.9811577675836506e-07)]

Text: the united states branch also reports that at least four of our brothers ' homes sustained minor damage , along with two kingdom halls .

Explanation: [(np.str_('branch'), -5.933460915984073e-07), (np.str_('states'), -3.4537900879159076e-07), (np.str_('damage'), -2.7224922392537005e-07), (np.str_('sustained'), -2.1991929394524682e-07), (np.str_('united'), 1.398518223856316e-07), (np.str_('minor'), 1.2390038876121556e-07)]

Text: additionally , the storms caused major damage to a brother 's business property .

Explanation: [(np.str_('storms'), 2.204062891933905e-06), (np.str_('caused'), 1.3290147107268105e-06), (np.str_('property'), -1.2296395244240085e-06), (np.str_('major'), 8.47281350361927e-07), (np.str_('additionally'), 8.142361750747797e-07), (np.str_('the'), 5.82157035706973e-07)]

Text: local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster .

Explanation: [(np.str_('the'), -1.1127893285469542e-06), (np.str_('affected'), -8.56214137031255e-07), (np.str_('elders'), 6.003230312436369e-07), (np.str_('are'), -4.897385120279575e-07), (np.str_('disaster'), -4.518079582280951e-07), (np.str_('those'), 3.692731344800294e-07)]

Text: we know that our heavenly father , jehovah , is providing comfort to our brothers and sisters who are grieving because of this tragedy .

Explanation: [(np.str_('we'), -2.2797552963155086e-06), (np.str_('jehovah'), 1.8767449909779077e-06), (np.str_('heavenly'), 1.7188782326562614e-06), (np.str_('father'), -1.1991664092050441e-06), (np.str_('that'), -1.1012362201245365e-06), (np.str_('our'), -1.0849781962230887e-06)]

Text: international government agencies and officials have responded to russia 's supreme court decision that criminalizes the worship of jehovah 's witnesses in russia .

Explanation: [(np.str_('international'), 2.106508942119281e-06), (np.str_('agencies'), -1.484781134968309e-06), (np.str_('officials'), -5.026655486406757e-07), (np.str_('have'), -3.5857592535213093e-07), (np.str_('criminalizes'), -3.36121967682946e-07), (np.str_('russia'), -3.239006792986295e-07)]

Text: these statements have criticized russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity .

Explanation: [(np.str_('have'), -1.742298488112012e-06), (np.str_('statements'), -1.3635646063897632e-06), (np.str_('against'), -1.07635960249973e-06), (np.str_('religious'), 1.000110544649922e-06), (np.str_('known'), 6.708483954050872e-07), (np.str_('action'), -4.914444016665408e-07)]
```
**2 - Bảng dịch với mô hình đa nguồn**


```
'Brother Albert Barnett and his wife , Sister Susan Barnett , from the West Congregation in Tuscaloosa , Alabama'

Explanation: [('Sister', -9.458128371254788e-09), ('บาร', -6.983643577817748e-09), ('Alabama', 6.586893231817712e-09), ('กรเวสต', -6.581261933728665e-09), ('Brother', -5.165914074923474e-09), ('ภรรยาของเขา', 4.9249516469403836e-09)]


'Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12 , 2020 .'

Explanation: [('11', -5.190822749666012e-09), ('นท', -2.3475847830633787e-09), ('12', -2.341220547561406e-09), ('อว', -2.3026387707626902e-09), ('ทางตอนใต', -2.1318767306985813e-09), ('ฐอเมร', -1.965338952273433e-09)]


'Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .'

Explanation: [('ฝนตกหน', 1.4690527684446525e-08), ('างหน', 1.1838489693480623e-08), ('นาโดหลายล', -1.1380190342667764e-08), ('กสร', -9.238325418033572e-09), ('states', -9.130078379690171e-09), ('ฐ', -7.427503961791758e-09)]


'Sadly , Brother Albert Barnett and his wife , Sister Susan Barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .'

Explanation: [('were', -1.5241277705014675e-08), ('when', 1.4054316301066879e-08), ('Brother', -9.220822799161447e-09), ('มบ', -8.042452088677718e-09), ('struck', -6.69853220905953e-09), ('ภรรยาของเขา', -6.546856373500485e-09)]


'The United States branch also reports that at least four of our brothers' homes sustained minor damage , along with two Kingdom Halls .'

Explanation: [('The', -6.928235021465077e-09), ('four', -5.071216994084436e-09), ('าบ', -4.432967830896423e-09), ('อมก', -4.232161543104857e-09), ('านของบราเดอร', -3.150263066849767e-09), ('at', 3.076680514867252e-09)]


'Additionally , the storms caused major damage to a brother 's business property .'

Explanation: [('Additionally', -1.2439928725187752e-08), ('นอกจากน', -6.636375304280142e-09), ('จของบราเดอร', 3.356132293376232e-09), ('างความเส', 2.7879937253163163e-09), ('to', -2.7696949733876353e-09), ('กต', 2.668694757313025e-09)]


'Local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster .'

Explanation: [('ผ', -3.1315526823673458e-09), ('are', -2.33613591401886e-09), ('by', -2.308389847709204e-09), ('แลภาคกำล', -2.015262092676841e-09), ('บ', -1.883112631471506e-09), ('those', -1.8193213500220773e-09)]


'We know that our heavenly Father , Jehovah , is providing comfort to our brothers and sisters who are grieving because of this tragedy .'

Explanation: [('เราร', -4.7995118168464256e-09), ('We', -3.4862006587222655e-09), ('ดาบนสวรรค', -3.1818698508450587e-09), ('providing', 2.6072726160661064e-09), ('are', -2.4018433614400314e-09), ('พระยะโฮวา', -2.0913350460722445e-09)]


'International government agencies and officials have responded to Russia 's Supreme Court decision that criminalizes the worship of Jehovah 's Witnesses in Russia .'

Explanation: [('decision', -3.2014751213721923e-09), ('วยงานร', -3.164850234714558e-09), ('างประเทศและเจ', -2.8643337867162418e-09), ('หน', -2.030721277986848e-09), ('that', -1.8602313934959648e-09), ('s', 1.5595094341127537e-09)]


'These statements have criticized Russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity .'

Explanation: [('These', -1.071552542889616e-08), ('นการทางกฎหมายท', -3.2609704978947955e-09), ('have', -2.3760834263332615e-09), ('จกรรมทางศาสนาอย', -2.2072951349011548e-09), ('าน', -2.1120428582240097e-09), ('known', -1.595341121168384e-09)]

```
**3- bảng dịch mô hình với tăng cường dữ liệu**



```
Text: brother albert barnett and his wife , sister susan barnett , from the west congregation in tuscaloosa , alabama
Explanation: [(np.str_('barnett'), -1.3103550077611215e-06), (np.str_('albert'), 5.469086526697135e-07), (np.str_('alabama'), 4.5520018796542316e-07), (np.str_('west'), 4.4683578730688826e-07), (np.str_('and'), 4.322644132458623e-07), (np.str_('wife'), -3.88384079256479e-07)]

Text: severe storms ripped through parts of the southern and midwestern united states on january 11 and 12 , 2020 .
Explanation: [(np.str_('ripped'), -8.868815222356579e-07), (np.str_('2020'), 6.488173749100982e-07), (np.str_('storms'), -4.00397595392644e-07), (np.str_('united'), -3.567413575908317e-07), (np.str_('through'), -3.0474432974280877e-07), (np.str_('11'), -2.847094985820839e-07)]

Text: two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .
Explanation: [(np.str_('rain'), -5.601602716887243e-07), (np.str_('tornadoes'), -3.3959246754658786e-07), (np.str_('two'), 3.215680273885648e-07), (np.str_('winds'), -3.062746427130424e-07), (np.str_('states'), -3.012394109016833e-07), (np.str_('and'), -2.0028454865215795e-07)]

Text: sadly , brother albert barnett and his wife , sister susan barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .
Explanation: [(np.str_('barnett'), -8.857721805859132e-07), (np.str_('brother'), -6.581881418682591e-07), (np.str_('and'), -6.343994998729984e-07), (np.str_('sadly'), -4.654398700272274e-07), (np.str_('respectively'), 4.21378937110256e-07), (np.str_('85'), 3.700906918938194e-07)]

Text: the united states branch also reports that at least four of our brothers ' homes sustained minor damage , along with two kingdom halls .
Explanation: [(np.str_('united'), -1.7571744865956402e-06), (np.str_('reports'), 1.7421208953549942e-06), (np.str_('our'), -1.511480569437083e-06), (np.str_('also'), -8.769838676639941e-07), (np.str_('minor'), -6.293829626347243e-07), (np.str_('states'), 5.348122244476235e-07)]

Text: additionally , the storms caused major damage to a brother 's business property .
Explanation: [(np.str_('brother'), -2.5373636971506774e-06), (np.str_('additionally'), -1.219839073024932e-06), (np.str_('business'), -4.642567523436242e-07), (np.str_('damage'), 3.3476036495261816e-07), (np.str_('major'), 2.4639217798821093e-07), (np.str_('s'), -1.3962044056946802e-07)]

Text: local elders and the circuit overseer are offering practical and spiritual support to those affected by this disaster .
Explanation: [(np.str_('local'), -6.048193064485463e-07), (np.str_('affected'), -5.542455448828236e-07), (np.str_('elders'), 4.855095122660242e-07), (np.str_('practical'), -3.3345427145339693e-07), (np.str_('disaster'), -3.332421241018685e-07), (np.str_('spiritual'), -2.3624210721300785e-07)]

Text: we know that our heavenly father , jehovah , is providing comfort to our brothers and sisters who are grieving because of this tragedy .
Explanation: [(np.str_('our'), -5.653883645200948e-07), (np.str_('we'), -2.3927145353669736e-07), (np.str_('brothers'), -1.6097107756701561e-07), (np.str_('sisters'), -1.5712924930123118e-07), (np.str_('know'), -1.2203769919555944e-07), (np.str_('providing'), 1.1717898556779603e-07)]

Text: international government agencies and officials have responded to russia 's supreme court decision that criminalizes the worship of jehovah 's witnesses in russia .
Explanation: [(np.str_('russia'), -2.654086994297247e-07), (np.str_('responded'), -2.421786646846226e-07), (np.str_('jehovah'), -1.8693937657856002e-07), (np.str_('worship'), -1.6976599449322918e-07), (np.str_('international'), -1.419870218950119e-07), (np.str_('witnesses'), -1.3835899703034013e-07)]

Text: these statements have criticized russia 's unjust and harsh judicial action against a minority religious group known for peaceful religious activity .
Explanation: [(np.str_('these'), 1.5916717223585013e-06), (np.str_('statements'), 5.306968746908933e-07), (np.str_('judicial'), 4.1164408197634823e-07), (np.str_('russia'), -3.361183415083695e-07), (np.str_('unjust'), -3.3371925571982056e-07), (np.str_('harsh'), -3.254668421905307e-07)]

```